In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy
import mpmath
from scipy import stats
from sklearn.neighbors import KernelDensity
import functools
import pandas as pd
import math
import random
import seaborn as sns
import itertools
from itertools import groupby
#import separation_algorithm as sepa
#import functions_eye_tracker_project as funcs
import sklearn
import os
from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LinearRegression
import copy
import emd

In [ ]:
def r_square(l_emp_points,l_emp_fit):
    l_num = np.mean((np.array(l_emp_points)-np.array(l_emp_fit))**2)
    l_den = np.std(np.array(l_emp_points))**2
    return(1 - l_num/l_den)


def adjusted_r_square(l_emp_points,l_emp_fit,degrees_freedom):
    n = len(l_emp_points)
    l_num = np.mean((np.array(l_emp_points)-np.array(l_emp_fit))**2)
    l_den = np.std(np.array(l_emp_points))**2
    rsqu = 1 - l_num/l_den
    return( 1 - (1 -rsqu)*( (n-1)/(n-degrees_freedom)  ))
    
    
    
def powerl_fit(l_tau,l_k,l_a):
    return(l_k*np.power(2,l_tau*l_a))

def Expr_variable_change(l_alpha,l_beta,D,v):
    l_lambdaD = l_beta(1-l_alpha)
    l_lambdaB = l_alpha*l_beta
    return(l_lambdaD,l_lambdaB,D,v)

def Expr_variable_change2(l_lambdaD,l_lambdaB,D,v):
    l_beta = l_lambdaB + l_lambdaD
    l_alpha = l_lambdaB/(l_beta)    
    return(l_alpha,l_beta,D,v)

def first_estimate_ple_averages_inc(l_emp_points,l_tau_list):
    if l_tau_list[1] != 2 * l_tau_list[0]:
        print("please use consecutive taus")
    if l_tau_list[2] != 2 * l_tau_list[1]:
        print("please use consecutive taus")
    
    lmbda_b = -(np.log(np.sqrt( (-l_emp_points[2] + 2*l_emp_points[1])/(-l_emp_points[1] + 2*l_emp_points[0])  ) - 1))/(l_tau_list[0])
    lk = np.exp(-lmbda_b*l_tau_list[0])
    
    C11 =  (-l_emp_points[1] + 2*l_emp_points[0]) /(1-lk)**2
    C12 =  (-l_emp_points[2] + 2*l_emp_points[1]) /(1-lk**2)**2
    l_C1 = (C11 + C12)/2
    
    C21 = (l_emp_points[0] - l_C1*(1-lk))/(l_tau_list[0])
    C22 = (l_emp_points[1] - l_C1*(1-lk**2))/(2*l_tau_list[0])
    C23 = (l_emp_points[2] - l_C1*(1-lk**4))/(4*l_tau_list[0])
    
    l_C2 = (C21 + C22 + C23)/3
    l_valid = 0
    if lmbda_b>0:
        if l_C2>0:
            l_valid = 1
    
    return([l_C1,l_C2,lmbda_b],l_valid)

def optimize(l_emp_points,l_tau_list,l_p0,max_iter,cutoff):
    init_ratio = 0.8
    C1_i,C2_i,lm_i = l_p0
    l_emp_points = np.log2(l_emp_points)
    log_tau_list = np.log2(l_tau_list)
    tau_log_incr =np.diff(log_tau_list)
    
    first_min_found = False
    
    if np.sum(tau_log_incr!=1)>0:
        print("tau list should be exponential with base 2")
#    print(l_emp_points)
    
    for i in range(max_iter):
        ratio = init_ratio+1
        C1_i,C2_i,lm_i =  C1_i/ratio,C2_i/ratio,lm_i/ratio
        error_list = []
        for factor1 in range(3):
            for factor2 in range(3):
                for factor3 in range(3):    
                    new_fit = log2_moment_scaling(l_tau_list,C1_i*(ratio**factor1),C2_i*(ratio**factor2),lm_i*(ratio**factor3))
                    new_fit[np.isnan(new_fit)] = 0
                    error_list.append(np.mean(( np.array(new_fit)-l_emp_points )**2))
#                    if factor1==factor2==factor3==1:
#                        print(log2_moment_scaling(l_tau_list,C1_i*(ratio**factor1),C2_i*(ratio**factor2),lm_i*(ratio**factor3)))
#                        print(C1_i*(ratio**factor1),C2_i*(ratio**factor2),lm_i*(ratio**factor3))
        
#        print(error_list)
        
        index_opt = np.argmin(error_list)
        opt_f1 = int(index_opt/9)
        opt_f2 = int( (index_opt%9)/3 )
        opt_f3 = int( index_opt%3 )
        C1_i = C1_i*(ratio**opt_f1)
        C2_i = C2_i*(ratio**opt_f2)
        lm_i = lm_i*(ratio**opt_f3)
#        print(opt_f1,opt_f2,opt_f3)
        
        if index_opt == 13:
            init_ratio = init_ratio/1.9
            first_min_found = True
        else:
            if not first_min_found:
                init_ratio = 1.9*init_ratio
                if i>cutoff:
                    return(C1_i,C2_i,lm_i)
                
    return(C1_i,C2_i,lm_i)

def Sergyi_expr_simp1(l_tau,l_alpha,l_beta,D,v):
    
    C1 = 2*((v/l_beta)**2) * (l_alpha - 1)/(l_alpha**2)
    C2 = 2*((1-l_alpha)/l_alpha) * ((v**2)/l_beta) + 4*D*l_alpha
    
    return(C1*(1-np.exp(-l_alpha*l_beta*l_tau)) + C2*l_tau )

def moment_scaling(tau_l,C1,C2,l_lambdaB):
    return(C1*(np.ones(len(tau_l))-np.exp(-l_lambdaB*np.array(tau_l)))  + C2*np.array(tau_l) )

def log2_moment_scaling(tau_l,C1,C2,l_lambdaB):
    return(np.log2(C1*(np.ones(len(tau_l))-np.exp(-l_lambdaB*np.array(tau_l)))  + C2*np.array(tau_l) ))

    
    
    
def C2_new2(l_alpha,l_beta,D,v):
    C2 = 2*((1-l_alpha)/l_alpha) * ((v**2)/l_beta) + 2*D*l_alpha
    return(C2)
    
def C3_new2(l_alpha,l_beta,D,v):
    C3 = 2*((1-l_alpha)/(l_alpha**2)) *((v/l_beta)**2)
    return(C3)
def C1_new2(l_alpha,l_beta,D,v):
    C1 = 2*((v/l_beta)**2) * (l_alpha - 1)/(l_alpha**2)
    return(C1)


def intermittent2(nt,dt,mean_bal_sac,diffusion,rate12,rate21):

    P1 = rate21/(rate12+rate21)
    if np.random.random()<P1:
        regime=1
        waitt   = -math.log(1.0-np.random.uniform(0.0,1.0,None))/rate12
    else:
        regime=2
        angle2 = np.random.randint(2)*math.pi
        waitt   = -math.log(1.0-np.random.uniform(0.0,1.0,None))/rate21

    dts = math.sqrt(dt)
    x = np.zeros(nt)
    y = np.zeros(nt)
    time_since_last_jump = 0
    for i in range(1,nt):
        angle = random.random()*2*math.pi
        time_since_last_jump += dt        
        if regime == 1:
            #diffu = diffusion*np.random.normal(0,1)*dts
            dx = diffusion*np.random.normal(0,1)*dts
            dy = diffusion*np.random.normal(0,1)*dts
            x[i] = x[i-1] + dx
            y[i] = y[i-1] + dy
            if time_since_last_jump> waitt:
                waitt   = -math.log(1.0-np.random.uniform(0.0,1.0,None))/rate21
                regime = 2
                angle2 = angle
                time_since_last_jump =0
        if regime == 2:
            angle3 = angle2 
            bal = mean_bal_sac*dt 
            dx = bal*math.cos(angle3)
            dy = bal*math.sin(angle3)
            x[i] = x[i-1] + dx
            y[i] = y[i-1] + dy

            if time_since_last_jump> waitt:
                waitt   = -math.log(1.0-np.random.uniform(0.0,1.0,None))/rate12
                time_since_last_jump =0
                regime = 1
        
    return(x,y)

def levy_flight_2D_2(n_redirections,n_max,lalpha,tmin,measuring_dt):
    if lalpha <= 1:
        print("alpha should be larger than 1")
        return("alpha should be larger than 1")
    

    t_redirection= tmin*(np.ones(n_redirections) - np.random.rand(n_redirections))**(1.0/(-lalpha+1))   
    cum_t_redirection = np.cumsum(t_redirection)


    angle = np.random.rand(len(t_redirection))*2*math.pi
    x_increments = t_redirection*np.cos(angle)
    y_increments = t_redirection*np.sin(angle)
    l_x_list = np.cumsum(x_increments)
    l_y_list = np.cumsum(y_increments)
    

    if n_max*measuring_dt < cum_t_redirection[-1]:

        x_measured = np.interp(np.arange(0,n_max*measuring_dt,measuring_dt),np.cumsum(t_redirection),l_x_list)
        y_measured = np.interp(np.arange(0,n_max*measuring_dt,measuring_dt),np.cumsum(t_redirection),l_y_list)
        
    else:
        
        n_max = int(cum_t_redirection[-1]/measuring_dt)
        #print("me<zasuring time greater than simulated time. n_max becomes " + str(n_max))
        x_measured = np.interp(np.arange(0,n_max*measuring_dt,measuring_dt),np.cumsum(t_redirection),l_x_list)
        y_measured = np.interp(np.arange(0,n_max*measuring_dt,measuring_dt),np.cumsum(t_redirection),l_y_list)
        #print("measuring time greater than simulated time.")
    return x_measured,y_measured,t_redirection


In [ ]:
def angle_two_vector(ldx,ldy):
    if np.any(np.isnan(ldx)):
        print("nan in input vector")
        return("nan in input vector")
    if np.any(np.isnan(ldy)):
        print("nan in input vector")
        return("nan in input vector")
    if len(ldx) != len(ldy):
        print("len of vectors should be the same")
        return( "len of vectors should be the same")
    angle_vec = []
    ldx_1 = np.array(ldx[1:])
    ldx_2 = np.array(ldx[:-1])
    ldy_1 = np.array(ldy[1:])
    ldy_2 = np.array(ldy[:-1])
    angle_vec1 = np.angle(ldx_1 + ldy_1*1.0j,deg = True)
    angle_vec2 = np.angle(ldx_2 + ldy_2*1.0j,deg = True)

    angle_diff = angle_vec2-angle_vec1

    angle_diff = angle_diff + 360*np.ones(len(angle_diff))*np.array(angle_diff<0) - 360*np.ones(len(angle_diff))*np.array(angle_diff>360) 
 

    return(angle_diff)

def angle_single_vector(ldx,ldy):
    if np.any(np.isnan(ldx)):
        print(  "nan in input vector")
        return( "nan in input vector")
    if np.any(np.isnan(ldy)):
        print( "nan in input vector")
        return( "nan in input vector")
    if len(ldx) != len(ldy):
        print( "len of vectors should be the same")
        return( "len of vectors should be the same")
    angle_vec = []
    ldx = np.array(ldx)
    ldy = np.array(ldy)
    angle_vec = np.angle(ldx + ldy*1.0j,deg = True)

    return( angle_vec)

In [ ]:
import os
import numpy as np

current_directory = os.getcwd()
print(current_directory)

# Define the directory path
directory_path = '/Users/pedroreg/Desktop/research copy/Flicker frequency'

#data_F0 = [] #no flickering detected
#data_F1 = [] #60 hz flickering detected
#data_F2 = [] #120 hz flickering detected

group00 = []
group10 = []
group11 = []
all_data = []

for filename in os.listdir(directory_path):
    # Construct the full file path
    file_path = os.path.join(directory_path, filename)
    if 'csv' in filename:
        if os.path.isfile(file_path):
            try:
                if 'T120_1' in file_path:
                    all_data.append(pd.read_csv(file_path))
                    group00.append(0)
                    group10.append(1)
                    group11.append(1)
                elif 'T60_1' in file_path:
                    all_data.append(pd.read_csv(file_path))
                    group00.append(0)
                    group10.append(1)
                    group11.append(0)
                elif 'T60_0'in file_path:
                    all_data.append(pd.read_csv(file_path))
                    group00.append(1)
                    group10.append(0)
                    group11.append(0)
                else:
                    print('ERROR, NO F')
                #print(f'Data in {filename}:\n{data}\n')
            except Exception as e:
                print(f'Error reading {filename}: {e}')
group00 = np.array(group00).astype('bool')
group10 = np.array(group10).astype('bool')
group11 = np.array(group11).astype('bool')

In [ ]:
T = []
X = []
Y = []
d_X = []
d_Y = []
XX = []
YY = []
TT = []
d_T = []
d_S = []
dT = []
dX = []
dY = []
dS = []
saccade = []
Ssaccade = []
vS = []
remove_nanbefore = 200

            

for i in range(0,len(all_data)):
    cont = 0


    T.append(np.swapaxes(np.array(all_data[i]),0,1)[0])
    X.append(np.swapaxes(np.array(all_data[i]),0,1)[1])
    Y.append(np.swapaxes(np.array(all_data[i]),0,1)[2])
    saccade.append(np.swapaxes(np.array(all_data[i]),0,1)[3])
    
    d_X.append(np.diff(X[-1]))
    d_Y.append(np.diff(Y[-1]))
    d_T.append(np.diff(T[-1]))
    d_S.append(np.sqrt(np.diff(X[-1])**2 + np.diff(Y[-1])**2))

    #remove NaN
    Nan_array = np.array(np.logical_not(np.isnan(d_X[-1])))*np.array(np.logical_not(np.isnan(d_Y[-1])))
    new_nan_array = copy.copy(Nan_array)
    for i in range(1,remove_nanbefore):
        new_nan_array = new_nan_array[:-1]*Nan_array[i:]
    Nan_array = new_nan_array
    
    for i in range(1,remove_nanbefore):
        new_nan_array = new_nan_array[1:]*Nan_array[:-i]
    Nan_array = new_nan_array
    Nan_array = np.append(Nan_array,np.zeros(remove_nanbefore-1))
    Nan_array = np.append(np.zeros(remove_nanbefore-1),Nan_array)
    Nan_array = Nan_array.astype(bool)


    dX.append(d_X[-1][Nan_array])
    dY.append(d_Y[-1][Nan_array])
    dS.append(np.sqrt(np.array(dX[-1])**2 + np.array(dY[-1])**2))
    dT.append(d_T[-1][Nan_array])
    vS.append(np.array(dS[-1])/np.array(dT[-1]))
    XX.append(np.array(X[-1][:-1])[Nan_array])
    YY.append(np.array(Y[-1][:-1])[Nan_array])
    Ssaccade.append(np.array(saccade[-1][:-1])[Nan_array])


In [ ]:
def power_spectrum(vector,frequency,graph,label):
    local_FT = np.fft.rfft(vector)
    local_P_S = np.square(np.abs(local_FT))
    frequencies = np.linspace(0, frequency/2, len(local_P_S))
    if graph:
        plt.plot(frequencies,local_P_S)
        #plt.plot(local_P_S)
        plt.title("Power Spectrum of " + label)
        plt.xlabel("frequency (HZ)")
        #plt.savefig("power_spectrum"+label+".png",dpi=500)
        plt.show()
        plt.close()
    return( local_P_S )

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 16})

In [ ]:

Greater = np.array([1,np.sum(group00),np.sum(group11==0)] )/81
Lesser = np.array([80,np.sum(group10),np.sum(group11)] )/81
  
spacing2 = 0.3
r = np.arange(0,3*spacing2,spacing2) 
width = 0.2
spacing = 0.1

#014E20
#02B07B

color_palette = ['#014E20','#02B07B','#01204E','#028391','#811638','#FAA968']
  
#plt.bar(r, Greater, color = [color_palette[0],color_palette[2],color_palette[4]], 
#        width = width, edgecolor = 'black') 
plt.bar(r + spacing + width, Lesser, color = [color_palette[1],color_palette[3],color_palette[5]], 
        width = width, edgecolor = 'black') 

plt.xticks([width+spacing,spacing2+width+spacing,2*spacing2+0.5*width + 2*spacing] ,[r'$\mathcal{P}_{>30}$',r'$\mathcal{P}_{>60}$',r'$\mathcal{P}_{>120}$']) 
  
plt.ylabel('Fraction of Participants')
plt.savefig('Nr_participants5.png',dpi=300)
  
# plt.grid(linestyle='--') 
#plt.xticks(r + width/2,['2018','2019','2020','2021']) 
  
plt.show() 

In [ ]:
Phi_two_vector_tot = []
Phi_two_vector_less_60 = []
Phi_two_vector_greater_60 = []
Phi_two_vector_less_120 = []
Phi_two_vector_greater_120 = []
for i in range(1,50):
    #print(i)
    Phi_two_vector_tot.append([])
    Phi_two_vector_less_60.append([])
    Phi_two_vector_greater_60.append([])
    Phi_two_vector_less_120.append([])
    Phi_two_vector_greater_120.append([])
    for k in range(len(dX)):
        Phi_two_vector_tot[-1].append(angle_two_vector(dX[k][::i],dY[k][::i]))
        if k in np.where(group00)[0]:
            Phi_two_vector_less_60[-1].append(angle_two_vector(dX[k][::i],dY[k][::i]))
        if k in np.where(group10)[0]:
            Phi_two_vector_greater_60[-1].append(angle_two_vector(dX[k][::i],dY[k][::i]))
        if k in np.where(group11==0)[0]:
            Phi_two_vector_less_120[-1].append(angle_two_vector(dX[k][::i],dY[k][::i]))
        if k in np.where(group11)[0]:
            Phi_two_vector_greater_120[-1].append(angle_two_vector(dX[k][::i],dY[k][::i]))

    Phi_two_vector_tot[-1] = np.hstack(Phi_two_vector_tot[-1])
    Phi_two_vector_less_60[-1] = np.hstack(Phi_two_vector_less_60[-1])
    Phi_two_vector_greater_60[-1] = np.hstack(Phi_two_vector_greater_60[-1])
    Phi_two_vector_less_120[-1] = np.hstack(Phi_two_vector_less_120[-1])
    Phi_two_vector_greater_120[-1] = np.hstack(Phi_two_vector_greater_120[-1])
        
    

In [ ]:
lranges = [360/2,360/4,360/8]
lcolors = ['#387FC8', '#F3B552','#393943' ]
labels = [r'$k=2$', '$k=4$','$k=8$' ]
lenarr = 10

PPtot_list = []

for l in range(len(lranges)):

    PPtot = []
    #PPge120 = []
    #PPle120 = []
    #PPge60 = []
    #PPle60 = []
    #lrange = 180
    for i in np.arange(lenarr):
        PPtot.append(np.sum( (Phi_two_vector_tot[i]>180-0.5*lranges[l])*(Phi_two_vector_tot[i]<180+0.5*lranges[l]) )/len(Phi_two_vector_tot[i]))
        #PPge120.append(np.sum( (Phi_two_vector_greater_120[i]>180-0.5*lrange)*(Phi_two_vector_greater_120[i]<180+0.5*lrange) )/len(Phi_two_vector_greater_120[i]))
        #PPle120.append(np.sum( (Phi_two_vector_less_120[i]>180-0.5*lrange)*(Phi_two_vector_less_120[i]<180+0.5*lrange) )/len(Phi_two_vector_less_120[i]))
        #PPge60.append(np.sum( (Phi_two_vector_greater_60[i]>180-0.5*lrange)*(Phi_two_vector_greater_60[i]<180+0.5*lrange) )/len(Phi_two_vector_greater_60[i]))
        #PPle60.append(np.sum( (Phi_two_vector_less_60[i]>180-0.5*lrange)*(Phi_two_vector_less_60[i]<180+0.5*lrange) )/len(Phi_two_vector_less_60[i]))
    PPtot_list.append(PPtot)
    plt.plot(np.arange(1,lenarr+1),PPtot,c=lcolors[l],label = labels[l])
    plt.plot(np.arange(1,lenarr+1),PPtot,'.',c=lcolors[l])
    #plt.plot(np.arange(1,31),PPge120,c='#FAA968', label=r'$\mathcal{P}_{>120}$')
    plt.xlabel(r'$\tau$ (ms)')
    #plt.ylabel(r'$Pr(\phi \in [0.9\pi,1.1\pi]$')
    plt.ylabel(r'$P_{\phi,k}$',fontsize=20)
    plt.legend(loc='right')
    #plt.text(0, 0.4, r'$t^{\prime} = 2.5$ms', fontsize=15)
    #plt.text(3.55, 0.35, r'$t^{\prime} = 3.5$ms', fontsize=15)
    plt.text(3.05, 0.45, r'$\tau = 3$ms', fontsize=15)
    plt.vlines(2.5,-0,0.59,linestyle='dotted',color='dimgrey',alpha=0.5)
    plt.vlines(3.5,-0,0.59,linestyle='dotted',color='dimgrey',alpha=0.5)
    plt.vlines(3,-0,0.59,linestyle='--',color='dimgrey',alpha=0.5)
    plt.ylim([0,0.59])
    #plt.hlines(lrange/360,1,30,linestyle='--',color='dimgrey',alpha=0.5)
    plt.tight_layout()

plt.savefig('pr_phi_combo.png',dpi=300)

In [ ]:
textt = ["a)","b)","c)"]
for j in np.arange(3):
    i = [1,3,7][j]
    kde = stats.gaussian_kde(1.1*np.abs(np.abs(2*math.pi*(Phi_two_vector_tot[i])/360 -math.pi)-math.pi)-0.12,bw_method=0.1)
    bins = np.arange(0.05,math.pi-0.05,math.pi/100)
    plt.plot(bins,kde(bins),c='k')
    #plt.hist(np.abs(np.abs(2*math.pi*(Phi_two_vector_tot[i])/360 -math.pi)-math.pi),np.arange(-0.05*math.pi,1.05*math.pi,math.pi/20),density = True);
    print(i)
    plt.xlabel(r'$\phi$')
    plt.ylabel(r'Density')
    plt.title(r'$t^{\prime} = $'+str(i+1) + ' ms')
    plt.xticks(ticks=([0,math.pi/4,math.pi/2,0.75*math.pi,math.pi]) ,labels=(['0',r'$\pi/4$',r'$\pi/2$',r'$3\pi/4$',r'$\pi$']))
    plt.tight_layout()
    plt.text(2.8, max(kde(bins)), textt[j], ha='right', va='top', fontsize=20)
    plt.savefig('phi_density_t='+str(i+1)+'.png',dpi=300)
    plt.show()

In [ ]:

table = np.zeros([6,16]).astype(list) #varname, 60 frequency , 60 p-val, 120 frequency , 120 p-val,


In [ ]:
################## Using normalized velocity ########################

cum_list = []
Energy = []
N = 1
skip_lag = 1
frequency = 1000/skip_lag
i_f_normed_dx = []
f_normed_dx = []

for k in range(len(dX)):
    norm_dx = np.cos(np.array(angle_single_vector(np.diff(XX[k][::skip_lag]),np.diff(YY[k][::skip_lag])))*math.pi/180)
    l_P_S = power_spectrum(norm_dx,frequency,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))
    #plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()


    cummulative = np.cumsum(moving_averages)/np.sum(moving_averages)
    plt.plot(frequencies,cummulative)
    #plt.plot(frequencies,moving_averages)
    #print(cummulative[int( len(moving_averages)*60*skip_lag/frequency)])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.2)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.4)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.6)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.8)[0][0]])
    i_f_normed_dx.append([])
    f_normed_dx.append([])
    for fff in np.arange(0,1,0.001):
        i_f_normed_dx[-1].append(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>=fff)[0][0]])
        f_normed_dx[-1].append(cummulative[int(len(cummulative)*fff)])

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))
f_normed_dx = np.array(f_normed_dx)
i_f_normed_dx = np.array(i_f_normed_dx)

In [ ]:
s_i_f_normed_dx = np.swapaxes(i_f_normed_dx,0,1)
s_f_normed_dx = np.swapaxes(f_normed_dx,0,1)
pvals_i = []
pvals_f = []
for i in range(len(s_i_f_normed_dx)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_i_f_normed_dx[i][group00],s_i_f_normed_dx[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_f_normed_dx[i][group00],s_f_normed_dx[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(f_normed_dx[group00],axis=0)-np.mean(f_normed_dx[group10],axis=0)))

selected_i_f_normed_dx60 = s_i_f_normed_dx[index_fopt60]
selected_f_normed_dx60 = s_f_normed_dx[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][0] = 'Ps $v_{x,unit}$'
table[1][0] = index_fopt60*500*0.001
table[2][0] = pvals_f[index_fopt60]

###################### 120 hz #####################

s_i_f_normed_dx = np.swapaxes(i_f_normed_dx,0,1)
s_f_normed_dx = np.swapaxes(f_normed_dx,0,1)
pvals_i = []
pvals_f = []
for i in range(len(s_i_f_normed_dx)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_i_f_normed_dx[i][group00],s_i_f_normed_dx[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_f_normed_dx[i][group00],s_f_normed_dx[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(f_normed_dx[group00],axis=0)-np.mean(f_normed_dx[group11],axis=0)))

selected_i_f_normed_dx120 = s_i_f_normed_dx[index_fopt120]
selected_f_normed_dx120 = s_f_normed_dx[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][0] = index_fopt120*500*0.001
table[4][0] = pvals_f[index_fopt120]

In [ ]:
################## Using normalized velocity ########################


Energy = []
N = 1
skip_lag = 1
frequency = 1000/skip_lag
i_f_normed_dy = []
f_normed_dy = []

for k in range(len(dX)):
    norm_dy = np.sin(np.array(angle_single_vector(np.diff(XX[k][::skip_lag]),np.diff(YY[k][::skip_lag])))*math.pi/180)
    l_P_S = power_spectrum(norm_dy,frequency,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))
    #plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()

    

    cummulative = np.cumsum(moving_averages)/np.sum(moving_averages)
    plt.plot(frequencies,cummulative)
    #plt.plot(frequencies,moving_averages)
    #print(cummulative[int( len(moving_averages)*60*skip_lag/frequency)])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.2)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.4)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.6)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.8)[0][0]])
    i_f_normed_dy.append([])
    f_normed_dy.append([])
    for fff in np.arange(0,1,0.001):
        i_f_normed_dy[-1].append(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>=fff)[0][0]])
        f_normed_dy[-1].append(cummulative[int(len(cummulative)*fff)])

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))

In [ ]:
vec = f_normed_dy
ivec = i_f_normed_dy
table_index = 1
varname = 'Ps $v_{y,unit}$'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)

pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = s_vec[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################


pvals_i = []
pvals_f = []
for i in range(len(s_i_f_normed_dx)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
color_palette = ['#01204E','#028391','#811638','#FAA968']
index = index_fopt60

plt.errorbar(np.linspace(0,500,1000),np.mean(vec[group00],axis=0),yerr=np.std(vec[group00],axis=0)/np.sqrt(np.sum(group00)),c = color_palette[0], label=r'$\mathcal{P}_{<60}$')
plt.errorbar(np.linspace(0,500,1000),np.mean(vec[group10],axis=0),yerr=np.std(vec[group10],axis=0)/np.sqrt(np.sum(group10)),c = color_palette[1], label=r'$\mathcal{P}_{>60}$',alpha=1)
plt.xlabel(r'$f^{\prime}$ (Hz)',fontsize = 25)
plt.ylabel(r'$\langle C_{v_{y,\text{unit}}}(f^{\prime})\rangle$',fontsize = 25)
plt.vlines(index*500*0.001,0,1,linestyle='--',color='dimgrey')
plt.annotate(r'$f^*$',(index*500*0.001+5,0.05),fontsize = 25)
plt.legend()
plt.tight_layout()
plt.savefig('PS_C_group60.pdf',dpi=300,format="pdf", bbox_inches="tight" )
plt.show()

index = index_fopt120

bins = np.arange(0,0.3,0.001)
plt.errorbar(np.linspace(0,500,1000),np.mean(vec[group00],axis=0),yerr=np.std(vec[group00],axis=0)/np.sqrt(np.sum(group00)),c = color_palette[2], label=r'$\mathcal{P}_{<120}$')
plt.errorbar(np.linspace(0,500,1000),np.mean(vec[group11],axis=0),yerr=np.std(vec[group11],axis=0)/np.sqrt(np.sum(group11)),c = color_palette[3], label=r'$\mathcal{P}_{>120}$',alpha=1)
plt.vlines(index*500*0.001,0,1,linestyle='--',color='dimgrey')
plt.annotate(r'$f^*$',(index*500*0.001+5,0.05),fontsize = 25)
plt.xlabel(r'$f^{\prime}$ (Hz)',fontsize = 25)
plt.ylabel(r'$\langle C_{v_{y,\text{unit}}}(f^{\prime})\rangle$',fontsize = 25)
plt.legend()
plt.tight_layout()
plt.savefig('PS_C_group120.pdf',dpi=300,format="pdf", bbox_inches="tight" )
plt.show()

index = index_fopt60
color_palette = ['#01204E','#028391','#811638','#FAA968']
index = np.argmin(pvals_f)

kde00 = stats.gaussian_kde(s_vec[index][group00],bw_method=0.2)
kde10 = stats.gaussian_kde(s_vec[index][group10],bw_method=0.2)
#kde11 = stats.gaussian_kde(s_f_normed_dy[index][group11==1],bw_method=0.2)
bins = np.arange(0,0.4,0.001)
plt.plot(bins,kde00(bins),c = color_palette[0], label=r'$\mathcal{P}_{<60}$')
plt.plot(bins,kde10(bins),c = color_palette[1], label=r'$\mathcal{P}_{>60}$')
plt.ylabel('Density')
plt.xlabel(r'$C_{v_{y,\text{unit}}}(f^*=' + str(index_fopt60*500*0.001) +r'\text{Hz})$',fontsize = 21)
#plt.plot(bins,kde11(bins),c = color_palette[2], label=r'$\mathcal{P}_{>120}$')
plt.tight_layout()
plt.savefig('ps_group60.png',dpi=300)
plt.legend()
plt.show()


index = index_fopt120
color_palette = ['#01204E','#028391','#811638','#FAA968']
index = np.argmin(pvals_f)

kde11 = stats.gaussian_kde(s_vec[index][group11],bw_method=0.2)
kdeN11 = stats.gaussian_kde(s_vec[index][group00],bw_method=0.2)

bins = np.arange(0,0.4,0.001)
plt.plot(bins,kdeN11(bins),c = color_palette[2], label=r'$\mathcal{P}_{<120}$')
plt.plot(bins,kde11(bins),c = color_palette[3], label=r'$\mathcal{P}_{>120}$')
#plt.plot(bins,kde11(bins),c = color_palette[2], label=r'$\mathcal{P}_{>120}$')
plt.ylabel('Density')
plt.xlabel(r'$C_{v_{y,\text{unit}}}(f^*=' + str(index_fopt60*500*0.001) +r'\text{Hz})$',fontsize = 21)
plt.tight_layout()
plt.savefig('ps_group120.png',dpi=300)
plt.legend()
plt.show()

In [ ]:
s_vec[index][group11]

In [ ]:
################## Using fixation velocity ########################


Energy = []
N = 1
skip_lag = 1
frequency = 1000/skip_lag
i_f_fix_dx = []
f_fix_dx = []
count = 0
for k in range(len(dX)):
    if np.sum(Ssaccade[k])>0:
        norm_dx = dX[k][Ssaccade[k]==1]
    else:
        norm_dx = dX[k]
        count +=1
    l_P_S = power_spectrum(norm_dx,frequency,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))
    #plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()

    

    cummulative = np.cumsum(moving_averages)/np.sum(moving_averages)
    plt.plot(frequencies,cummulative)
    #plt.plot(frequencies,moving_averages)
    #print(cummulative[int( len(moving_averages)*60*skip_lag/frequency)])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.2)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.4)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.6)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.8)[0][0]])
    i_f_fix_dx.append([])
    f_fix_dx.append([])
    for fff in np.arange(0,1,0.001):
        i_f_fix_dx[-1].append(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>=fff)[0][0]])
        f_fix_dx[-1].append(cummulative[int(len(cummulative)*fff)])

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))

In [ ]:
vec = f_fix_dx
ivec = i_f_fix_dx
table_index = 2
varname = 'Ps $v_{x,fix}$'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)

pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = vec[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################


pvals_i = []
pvals_f = []
for i in range(len(s_i_f_normed_dx)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
['#01204E','#028391','#FAA968']

In [ ]:
################## Using fixation velocity ########################

Energy = []
N = 1
skip_lag = 1
frequency = 1000/skip_lag
i_f_fix_dy = []
f_fix_dy = []

for k in range(len(dX)):
    norm_dx = dY[k][Ssaccade[k]==1]
    if np.sum(Ssaccade[k])>0:
        norm_dx = dY[k][Ssaccade[k]==1]
    else:
        norm_dx = dY[k]
        count +=1
    l_P_S = power_spectrum(norm_dx,frequency,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))
    #plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()

    

    cummulative = np.cumsum(moving_averages)/np.sum(moving_averages)
    plt.plot(frequencies,cummulative)
    #plt.plot(frequencies,moving_averages)
    #print(cummulative[int( len(moving_averages)*60*skip_lag/frequency)])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.2)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.4)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.6)[0][0]])
    print(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.8)[0][0]])
    i_f_fix_dy.append([])
    f_fix_dy.append([])
    for fff in np.arange(0.001,1,0.001):
        i_f_fix_dy[-1].append(frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>=fff)[0][0]])
        f_fix_dy[-1].append(cummulative[int(len(cummulative)*fff)])

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))

In [ ]:
vec = f_fix_dy
ivec = i_f_fix_dy
table_index = 3
varname = 'Ps $v_{y,fix}$'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)

pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = s_vec[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################


pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
#############################################################################


In [ ]:

plt.rcParams.update({'font.size': 25})
imf = emd.sift.mask_sift(dY[1][Ssaccade[1]==1], max_imfs=4)

plt.figure(figsize=(4, 4))
emd.plotting.plot_imfs(imf[:1000*3, :])
plt.savefig('imf_example.png',dpi=300)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')

f, spectrum = emd.spectra.hilberthuang(IF, IA, (0, 300, 25))

freq_edges, freq_centres = emd.spectra.define_hist_bins(0, 300, 128, 'linear')

# Amplitude weighted HHT per IMF
f, spec_weighted = emd.spectra.hilberthuang(IF, IA, freq_edges, sum_imfs=False)

# Unweighted HHT per IMF - we replace the instantaneous amplitude values with ones
f, spec_unweighted = emd.spectra.hilberthuang(IF, np.ones_like(IA), freq_edges, sum_imfs=False)
lcolors = ['#1B9E77','#7570B3','#66A61E','#A6761D']


for i in range(4):
    norm_fac = np.sum((freq_centres[1]-freq_centres[0])*np.swapaxes(spec_unweighted,0,1)[i])
    plt.plot(freq_centres, np.array(np.swapaxes(spec_unweighted,0,1)[i])/norm_fac,c=lcolors[i])
plt.xticks(np.arange(5)*50)
plt.xlim(0, 300)
plt.xlabel(r'$\tilde{f}$ (Hz)')
plt.ylabel('Density')
plt.title('Instantaneous Frequency (IF)')
plt.legend([r'$\rho_{IMF1,v_{x,fix}}$', r'$\rho_{IMF2,v_{x,fix}}$', r'$\rho_{IMF3,v_{x,fix}}$', r'$\rho_{IMF4,v_{x,fix}}$'], frameon=False)
plt.tight_layout()
plt.savefig('HH-transform.png',dpi=300)

'''plt.subplot(122)
plt.plot(freq_centres, spec_weighted)
plt.xticks(np.arange(5)*50)
plt.xlim(0, 300)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.title('IA-weighted\nHilbert-Huang Transform')'''


In [ ]:
np.swapaxes(spec_unweighted,0,1)[0];

In [ ]:
#################################### IMF 1 - fix vel #########################

In [ ]:


i_f_emd_IF_dx = []
f_emd_IF_dx = []
i_f_emd_IA_dx = []
f_emd_IA_dx = []

for i in range(len(dX)):
    if np.sum(Ssaccade[i])>0:
        imf = emd.sift.mask_sift(dX[i][Ssaccade[i]==1], max_imfs=4)
    else:
        imf = emd.sift.mask_sift(dX[i], max_imfs=4)
    IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
    
    #emd.plotting.plot_imfs(imf[:1000*3, :])
    
    
    plt.figure(figsize=(8, 4))
    
    plt.subplot(121)
    # Plot a simple histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 150),color=lcolor);
    #peak_IF.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 0],bw_method=0.07)
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram')
    plt.xticks(np.arange(0, 500, 100))
    plt.xlabel('Frequency (Hz)')

    cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
    
    plt.subplot(122)
    # Plot an amplitude-weighted histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 100), weights=IA[:, 2],color=lcolor)
    #peak_IA.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 0],bw_method=0.07, weights=np.abs(IA[:, 2]))
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram\nweighted by IA')
    plt.xticks(np.arange(0, 500, 100))
    plt.xlabel('Frequency (Hz)')
    plt.show()
    
    cummulative_IA = np.cumsum(ghist)/np.sum(ghist)


    i_f_emd_IF_dx.append([])
    f_emd_IF_dx.append([])
    i_f_emd_IA_dx.append([])
    f_emd_IA_dx.append([])
    for fff in np.arange(0,1,0.001):
        i_f_emd_IF_dx[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
        f_emd_IF_dx[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
        i_f_emd_IA_dx[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
        f_emd_IA_dx[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])

In [ ]:
vec = f_emd_IF_dx
ivec = i_f_emd_IF_dx

table_index = 4
varname = 'IMF1 IF(v_{x,fix})'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)




pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = s_vec[index_fopt60]
print(index_fopt60,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################
pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
#################################### IMF 2 #########################

In [ ]:

i_f_emd_IF_dx = []
f_emd_IF_dx = []
i_f_emd_IA_dx = []
f_emd_IA_dx = []

for i in range(len(dX)):
    if np.sum(Ssaccade[i])>0:
        imf = emd.sift.mask_sift(dX[i][Ssaccade[i]==1], max_imfs=4)
    else:
        imf = emd.sift.mask_sift(dX[i], max_imfs=4)
    IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
    
    #emd.plotting.plot_imfs(imf[:1000*3, :])
    
    
    plt.figure(figsize=(8, 4))
    
    plt.subplot(121)
    # Plot a simple histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 150),color=lcolor);
    #peak_IF.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 1],bw_method=0.07)
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram')
    plt.xticks(np.arange(0, 500, 100))
    plt.xlabel('Frequency (Hz)')

    cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
    
    plt.subplot(122)
    # Plot an amplitude-weighted histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 100), weights=IA[:, 2],color=lcolor)
    #peak_IA.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 1],bw_method=0.07, weights=np.abs(IA[:, 2]))
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram\nweighted by IA')
    plt.xticks(np.arange(0, 500, 100))
    plt.xlabel('Frequency (Hz)')
    plt.show()
    
    cummulative_IA = np.cumsum(ghist)/np.sum(ghist)


    i_f_emd_IF_dx.append([])
    f_emd_IF_dx.append([])
    i_f_emd_IA_dx.append([])
    f_emd_IA_dx.append([])
    for fff in np.arange(0,1,0.001):
        i_f_emd_IF_dx[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
        f_emd_IF_dx[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
        i_f_emd_IA_dx[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
        f_emd_IA_dx[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])

In [ ]:
vec = f_emd_IF_dx
ivec = i_f_emd_IF_dx

table_index = 5
varname = 'IMF2 IF(v_{x,fix})'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)


pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = s_vec[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################
pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
#################################### IMF 3 fix vel #########################

In [ ]:
peak_IF = []
peak_IF60 = []
peak_IF60plus = []
peak_IA60 = []
peak_IA60plus = []
peak_IA = []

i_f_emd_IF_dx = []
f_emd_IF_dx = []
i_f_emd_IA_dx = []
f_emd_IA_dx = []

for i in range(len(dX)):
    if np.sum(Ssaccade[i])>0:
        imf = emd.sift.mask_sift(dX[i][Ssaccade[i]==1], max_imfs=4)
    else:
        imf = emd.sift.mask_sift(dX[i], max_imfs=4)
    IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
    
    #emd.plotting.plot_imfs(imf[:1000*3, :])
    
    
    plt.figure(figsize=(8, 4))
    
    plt.subplot(121)
    # Plot a simple histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 150),color=lcolor);
    #peak_IF.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 2],bw_method=0.07)
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    peak_IF.append( gbins[np.argmax(ghist)])
    peak_IF60.append(ghist[60])
    peak_IF60plus.append(np.sum(ghist[60:]))
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram')
    plt.xticks(np.arange(0, 150, 100))
    plt.xlabel('Frequency (Hz)')

    cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
    
    plt.subplot(122)
    # Plot an amplitude-weighted histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 100), weights=IA[:, 2],color=lcolor)
    #peak_IA.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 2],bw_method=0.07, weights=np.abs(IA[:, 2]))
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    peak_IA.append( gbins[np.argmax(ghist)])
    peak_IA60.append(ghist[60])
    peak_IA60plus.append(np.sum(ghist[60:]))
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram\nweighted by IA')
    plt.xticks(np.arange(0, 150, 100))
    plt.xlabel('Frequency (Hz)')
    plt.show()
    
    cummulative_IA = np.cumsum(ghist)/np.sum(ghist)


    i_f_emd_IF_dx.append([])
    f_emd_IF_dx.append([])
    i_f_emd_IA_dx.append([])
    f_emd_IA_dx.append([])
    for fff in np.arange(0,1,0.001):
        i_f_emd_IF_dx[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
        f_emd_IF_dx[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
        i_f_emd_IA_dx[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
        f_emd_IA_dx[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])

In [ ]:
vec = f_emd_IF_dx
ivec = i_f_emd_IF_dx

table_index = 6
varname = 'IMF3 IF(v_{x,fix})'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)


pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = s_vec[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################
pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]




In [ ]:
############################# IMF 1 - Y ############################

In [ ]:


i_f_emd_IF_dy = []
f_emd_IF_dy = []
i_f_emd_IA_dy = []
f_emd_IA_dy = []

for i in range(len(dX)):
    if np.sum(Ssaccade[i])>0:
        imf = emd.sift.mask_sift(dY[i][Ssaccade[i]==1], max_imfs=4)
    else:
        imf = emd.sift.mask_sift(dY[i], max_imfs=4)
    IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
    
    #emd.plotting.plot_imfs(imf[:1000*3, :])
    
    
    plt.figure(figsize=(8, 4))
    
    plt.subplot(121)
    # Plot a simple histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 150),color=lcolor);
    #peak_IF.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 0],bw_method=0.07)
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram')
    plt.xticks(np.arange(0, 500, 20))
    plt.xlabel('Frequency (Hz)')

    cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
    
    plt.subplot(122)
    # Plot an amplitude-weighted histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 100), weights=IA[:, 2],color=lcolor)
    #peak_IA.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 0],bw_method=0.07, weights=np.abs(IA[:, 2]))
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram\nweighted by IA')
    plt.xticks(np.arange(0, 500, 20))
    plt.xlabel('Frequency (Hz)')
    plt.show()
    
    cummulative_IA = np.cumsum(ghist)/np.sum(ghist)


    i_f_emd_IF_dy.append([])
    f_emd_IF_dy.append([])
    i_f_emd_IA_dy.append([])
    f_emd_IA_dy.append([])
    for fff in np.arange(0,1,0.001):
        i_f_emd_IF_dy[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
        f_emd_IF_dy[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
        i_f_emd_IA_dy[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
        f_emd_IA_dy[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])

In [ ]:
vec = f_emd_IF_dy
ivec = i_f_emd_IF_dy

table_index = 7
varname = 'IMF1 IF(v_{y,fix})'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)


pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = s_vec[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################
pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
color_palette = ['#01204E','#028391','#811638','#FAA968']
index = index_fopt60

plt.errorbar(np.linspace(0,500,1000),np.mean(vec[group00],axis=0),yerr=0.5*np.std(vec[group00],axis=0)/np.sqrt(np.sum(group00)),c = color_palette[0], label=r'$\mathcal{P}_{<60}$')
plt.errorbar(np.linspace(3,503,1000),np.mean(vec[group10],axis=0),yerr=0.5*np.std(vec[group10],axis=0)/np.sqrt(np.sum(group10)),c = color_palette[1], label=r'$\mathcal{P}_{>60}$',alpha=1)
plt.xlabel(r'$f^{\prime}$ (Hz)',fontsize = 25)
plt.ylabel(r'$\langle \text{cdf}_{v_{y,\text{fix}},S1}(f^{\prime})\rangle$',fontsize = 25)
plt.vlines(index*500*0.001,0,1,linestyle='--',color='dimgrey')
plt.annotate(r'$f^*$',(index*500*0.001+5,0.05),fontsize = 25)
plt.legend()
plt.tight_layout()
plt.savefig('emd_C_group60.pdf',dpi=300,format="pdf", bbox_inches="tight" )
plt.show()

index = index_fopt120

bins = np.arange(0,0.3,0.001)
plt.errorbar(np.linspace(0,500,1000),np.mean(vec[group00],axis=0),yerr=0.2*np.std(vec[group00],axis=0)/np.sqrt(np.sum(group00)),c = color_palette[2], label=r'$\mathcal{P}_{<120}$')
plt.errorbar(np.linspace(3,503,1000),np.mean(vec[group11],axis=0),yerr=0.2*np.std(vec[group11],axis=0)/np.sqrt(np.sum(group11)),c = color_palette[3], label=r'$\mathcal{P}_{>120}$',alpha=1)
plt.vlines(index*500*0.001,0,1,linestyle='--',color='dimgrey')
plt.annotate(r'$f^*$',(index*500*0.001+5,0.05),fontsize = 25)
plt.xlabel(r'$f^{\prime}$ (Hz)',fontsize = 25)
plt.ylabel(r'$\langle \text{cdf}_{v_{y,\text{fix}},S1}(f^{\prime})\rangle$',fontsize = 25)
plt.legend()
plt.tight_layout()
plt.savefig('emd_C_group120.pdf',dpi=300,format="pdf", bbox_inches="tight" )
plt.show()

index = index_fopt60
color_palette = ['#01204E','#028391','#811638','#FAA968']
index = np.argmin(pvals_f)

kde00 = stats.gaussian_kde(s_vec[index][group00],bw_method=0.2)
kde10 = stats.gaussian_kde(s_vec[index][group10],bw_method=0.2)
#kde11 = stats.gaussian_kde(s_f_normed_dy[index][group11==1],bw_method=0.2)
bins = np.arange(0.95,1,0.0001)
plt.plot(bins,kde00(bins),c = color_palette[0], label=r'$\mathcal{P}_{<60}$')
plt.plot(bins,kde10(bins),c = color_palette[1], label=r'$\mathcal{P}_{>60}$')
plt.ylabel('Density',fontsize = 21)
plt.xlabel(r'$\text{cdf}_{v_{y,\text{fix}},S1}(f^*=' + str(index_fopt60*500*0.001) +r'\text{Hz})$',fontsize = 21)
plt.tight_layout()
#plt.plot(bins,kde11(bins),c = color_palette[2], label=r'$\mathcal{P}_{>120}$')
plt.savefig('emd_group60.png',dpi=300)
plt.legend()
plt.show()


index = index_fopt120
color_palette = ['#01204E','#028391','#811638','#FAA968']
index = np.argmin(pvals_f)

kde11 = stats.gaussian_kde(s_vec[index][group11],bw_method=0.2)
kdeN11 = stats.gaussian_kde(s_vec[index][~group11],bw_method=0.2)

bins = np.arange(0.95,1,0.0001)
plt.plot(bins,kdeN11(bins),c = color_palette[2], label=r'$\mathcal{P}_{<120}$')
plt.plot(bins,kde11(bins),c = color_palette[3], label=r'$\mathcal{P}_{>120}$')
#plt.plot(bins,kde11(bins),c = color_palette[2], label=r'$\mathcal{P}_{>120}$')
plt.ylabel('Density',fontsize = 21)
plt.xlabel(r'$\text{cdf}_{v_{y,\text{fix}},S1}(f^*=' + str(index_fopt120*500*0.001) +r'\text{Hz})$',fontsize = 21)
plt.tight_layout()
plt.savefig('emd_group120.png',dpi=300)
plt.legend()
plt.show()

In [ ]:
############################# IMF 2 - Y ############################

In [ ]:


i_f_emd_IF_dy = []
f_emd_IF_dy = []
i_f_emd_IA_dy = []
f_emd_IA_dy = []

for i in range(len(dX)):
    if np.sum(Ssaccade[i])>0:
        imf = emd.sift.mask_sift(dY[i][Ssaccade[i]==1], max_imfs=4)
    else:
        imf = emd.sift.mask_sift(dY[i], max_imfs=4)
    IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
    
    #emd.plotting.plot_imfs(imf[:1000*3, :])
    
    
    plt.figure(figsize=(8, 4))
    
    plt.subplot(121)
    # Plot a simple histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 150),color=lcolor);
    #peak_IF.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 1],bw_method=0.07)
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram')
    plt.xticks(np.arange(0, 500, 20))
    plt.xlabel('Frequency (Hz)')

    cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
    
    plt.subplot(122)
    # Plot an amplitude-weighted histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 100), weights=IA[:, 2],color=lcolor)
    #peak_IA.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 1],bw_method=0.07, weights=np.abs(IA[:, 2]))
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram\nweighted by IA')
    plt.xticks(np.arange(0, 500, 20))
    plt.xlabel('Frequency (Hz)')
    plt.show()
    
    cummulative_IA = np.cumsum(ghist)/np.sum(ghist)


    i_f_emd_IF_dy.append([])
    f_emd_IF_dy.append([])
    i_f_emd_IA_dy.append([])
    f_emd_IA_dy.append([])
    for fff in np.arange(0,1,0.001):
        i_f_emd_IF_dy[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
        f_emd_IF_dy[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
        i_f_emd_IA_dy[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
        f_emd_IA_dy[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])

In [ ]:
vec = f_emd_IF_dy
ivec = i_f_emd_IF_dy

table_index = 8
varname = 'IMF2 IF(v_{y,fix})'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)


pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = s_vec[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################
pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
################################## IMF 3 - Y ########################

In [ ]:


i_f_emd_IF_dy = []
f_emd_IF_dy = []
i_f_emd_IA_dy = []
f_emd_IA_dy = []

for i in range(len(dX)):
    if np.sum(Ssaccade[i])>0:
        imf = emd.sift.mask_sift(dY[i][Ssaccade[i]==1], max_imfs=4)
    else:
        imf = emd.sift.mask_sift(dY[i], max_imfs=4)
    IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
    
    #emd.plotting.plot_imfs(imf[:1000*3, :])
    
    
    plt.figure(figsize=(8, 4))
    
    plt.subplot(121)
    # Plot a simple histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 150),color=lcolor);
    #peak_IF.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 2],bw_method=0.07)
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram')
    plt.xticks(np.arange(0, 150, 20))
    plt.xlabel('Frequency (Hz)')

    cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
    
    plt.subplot(122)
    # Plot an amplitude-weighted histogram using frequency bins from 0-20Hz
    #a = plt.hist(IF[:, 2], np.linspace(0, 100), weights=IA[:, 2],color=lcolor)
    #peak_IA.append(a[1][np.argmax(a[0])])
    kde = stats.gaussian_kde(IF[:, 2],bw_method=0.07, weights=np.abs(IA[:, 2]))
    gbins = np.arange(0,500,0.5)
    ghist = kde(gbins)
    plt.plot(gbins,ghist)
    plt.grid(True)
    plt.title('IF Histogram\nweighted by IA')
    plt.xticks(np.arange(0, 150, 20))
    plt.xlabel('Frequency (Hz)')
    plt.show()
    
    cummulative_IA = np.cumsum(ghist)/np.sum(ghist)


    i_f_emd_IF_dy.append([])
    f_emd_IF_dy.append([])
    i_f_emd_IA_dy.append([])
    f_emd_IA_dy.append([])
    for fff in np.arange(0,1,0.001):
        i_f_emd_IF_dy[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
        f_emd_IF_dy[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
        i_f_emd_IA_dy[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
        f_emd_IA_dy[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])




In [ ]:
vec = f_emd_IF_dy
ivec = i_f_emd_IF_dy

table_index = 9
varname = 'IMF3 IF(v_{y,fix})'

vec = np.array(vec)
ivec = np.array(ivec)
s_ivec = np.swapaxes(ivec,0,1)
s_vec = np.swapaxes(vec,0,1)


pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])

index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

selected_i_f_normed_dx60 = s_ivec[index_fopt60]
selected_f_normed_dx60 = s_vec[index_fopt60]
print(index_fopt60*500*0.001,pvals_f[index_fopt60])
#print(index_fopt*500*0.001,min(pvals_f))

table[0][table_index] = varname
table[1][table_index] = index_fopt60*500*0.001
table[2][table_index] = pvals_f[index_fopt60]

###################### 120 hz #####################
pvals_i = []
pvals_f = []
for i in range(len(s_ivec)):
    pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
    pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])

index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))

selected_i_f_normed_dx120 = s_ivec[index_fopt120]
selected_f_normed_dx120 = s_vec[index_fopt120]
print(index_fopt120*500*0.001,pvals_f[index_fopt120])
#print(index_fopt*500*0.001,min(pvals_f))


table[3][table_index] = index_fopt120*500*0.001
table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
################################# IMF 1-3 - v unit X #######################

In [ ]:
for iii in range(3):
    l_imf = iii
    table_index = 10+iii
    varname = 'IMF'+str(iii+1)+' IF(v_{x,unit})'
    
    i_f_emd_IF_dy = []
    f_emd_IF_dy = []
    i_f_emd_IA_dy = []
    f_emd_IA_dy = []
    
    for k in range(len(dX)):
        if k%10==0:
            print(k)
        
        vunit = np.cos(np.array(angle_single_vector(np.diff(XX[k][::skip_lag]),np.diff(YY[k][::skip_lag])))*math.pi/180)
        imf = emd.sift.mask_sift(vunit, max_imfs=4)
        IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
        
        #emd.plotting.plot_imfs(imf[:1000*3, :])
        
        
        plt.figure(figsize=(8, 4))
        
        plt.subplot(121)
        # Plot a simple histogram using frequency bins from 0-20Hz
        #a = plt.hist(IF[:, 2], np.linspace(0, 150),color=lcolor);
        #peak_IF.append(a[1][np.argmax(a[0])])
        kde = stats.gaussian_kde(IF[:, l_imf],bw_method=0.07)
        gbins = np.arange(0,500,0.5)
        ghist = kde(gbins)
        plt.plot(gbins,ghist)
        plt.grid(True)
        plt.title('IF Histogram')
        plt.xticks(np.arange(0, 150, 20))
        plt.xlabel('Frequency (Hz)')
    
        cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
        
        plt.subplot(122)
        # Plot an amplitude-weighted histogram using frequency bins from 0-20Hz
        #a = plt.hist(IF[:, 2], np.linspace(0, 100), weights=IA[:, 2],color=lcolor)
        #peak_IA.append(a[1][np.argmax(a[0])])
        kde = stats.gaussian_kde(IF[:, l_imf],bw_method=0.07, weights=np.abs(IA[:, 2]))
        gbins = np.arange(0,500,0.5)
        ghist = kde(gbins)
        plt.plot(gbins,ghist)
        plt.grid(True)
        plt.title('IF Histogram\nweighted by IA')
        plt.xticks(np.arange(0, 150, 20))
        plt.xlabel('Frequency (Hz)')
        #plt.show()
        plt.close()
        
        cummulative_IA = np.cumsum(ghist)/np.sum(ghist)
    
    
        i_f_emd_IF_dy.append([])
        f_emd_IF_dy.append([])
        i_f_emd_IA_dy.append([])
        f_emd_IA_dy.append([])
        for fff in np.arange(0,1,0.001):
            i_f_emd_IF_dy[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
            f_emd_IF_dy[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
            i_f_emd_IA_dy[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
            f_emd_IA_dy[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])
    
    
    ################################
    vec = f_emd_IF_dy
    ivec = i_f_emd_IF_dy
    
    vec = np.array(vec)
    ivec = np.array(ivec)
    s_ivec = np.swapaxes(ivec,0,1)
    s_vec = np.swapaxes(vec,0,1)
    
    
    pvals_i = []
    pvals_f = []
    for i in range(len(s_ivec)):
        pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
        pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])
    
    index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))
    
    selected_i_f_normed_dx60 = s_ivec[index_fopt60]
    selected_f_normed_dx60 = s_vec[index_fopt60]
    print(index_fopt60*500*0.001,pvals_f[index_fopt60])
    #print(index_fopt*500*0.001,min(pvals_f))
    
    table[0][table_index] = varname
    table[1][table_index] = index_fopt60*500*0.001
    table[2][table_index] = pvals_f[index_fopt60]
    
    ###################### 120 hz #####################
    pvals_i = []
    pvals_f = []
    for i in range(len(s_ivec)):
        pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
        pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])
    
    index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))
    
    selected_i_f_normed_dx120 = s_ivec[index_fopt120]
    selected_f_normed_dx120 = s_vec[index_fopt120]
    print(index_fopt120*500*0.001,pvals_f[index_fopt120])
    #print(index_fopt*500*0.001,min(pvals_f))
    
    
    table[3][table_index] = index_fopt120*500*0.001
    table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
s_vec[100][group00],s_vec[100][group10]

In [ ]:
np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))

In [ ]:
pvals_f

In [ ]:
f_emd_IF_dy[0]

In [ ]:
vec[0]

In [ ]:
s_vec[i][group00]

In [ ]:
for iii in range(3):
    l_imf = iii
    table_index = 13+iii
    varname = 'IMF'+str(iii+1)+' IF(v_{y,unit})'
    
    i_f_emd_IF_dy = []
    f_emd_IF_dy = []
    i_f_emd_IA_dy = []
    f_emd_IA_dy = []
    
    
    for k in range(len(dX)):
        if k%10==0:
            print(k)
        vunit = np.sin(np.array(angle_single_vector(np.diff(XX[k][::skip_lag]),np.diff(YY[k][::skip_lag])))*math.pi/180)
        imf = emd.sift.mask_sift(vunit, max_imfs=4)
        IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
        
        #emd.plotting.plot_imfs(imf[:1000*3, :])
        
        
        plt.figure(figsize=(8, 4))
        
        plt.subplot(121)
        # Plot a simple histogram using frequency bins from 0-20Hz
        #a = plt.hist(IF[:, 2], np.linspace(0, 150),color=lcolor);
        #peak_IF.append(a[1][np.argmax(a[0])])
        kde = stats.gaussian_kde(IF[:, l_imf],bw_method=0.07)
        gbins = np.arange(0,500,0.5)
        ghist = kde(gbins)
        plt.plot(gbins,ghist)
        plt.grid(True)
        plt.title('IF Histogram')
        plt.xticks(np.arange(0, 150, 20))
        plt.xlabel('Frequency (Hz)')
    
        cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
        
        plt.subplot(122)
        # Plot an amplitude-weighted histogram using frequency bins from 0-20Hz
        #a = plt.hist(IF[:, 2], np.linspace(0, 100), weights=IA[:, 2],color=lcolor)
        #peak_IA.append(a[1][np.argmax(a[0])])
        kde = stats.gaussian_kde(IF[:, l_imf],bw_method=0.07, weights=np.abs(IA[:, 2]))
        gbins = np.arange(0,500,0.5)
        ghist = kde(gbins)
        plt.plot(gbins,ghist)
        plt.grid(True)
        plt.title('IF Histogram\nweighted by IA')
        plt.xticks(np.arange(0, 150, 20))
        plt.xlabel('Frequency (Hz)')
        #plt.show()
        plt.close()
        
        cummulative_IA = np.cumsum(ghist)/np.sum(ghist)
    
    
        i_f_emd_IF_dy.append([])
        f_emd_IF_dy.append([])
        i_f_emd_IA_dy.append([])
        f_emd_IA_dy.append([])
        for fff in np.arange(0,1,0.001):
            i_f_emd_IF_dy[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
            f_emd_IF_dy[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
            i_f_emd_IA_dy[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
            f_emd_IA_dy[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])
    
    
    ################################
    vec = f_emd_IF_dy
    ivec = i_f_emd_IF_dy
    
    vec = np.array(vec)
    ivec = np.array(ivec)
    s_ivec = np.swapaxes(ivec,0,1)
    s_vec = np.swapaxes(vec,0,1)
    
    
    pvals_i = []
    pvals_f = []
    for i in range(len(s_ivec)):
        pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
        pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])
    
    index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))
    
    selected_i_f_normed_dx60 = s_ivec[index_fopt60]
    selected_f_normed_dx60 = s_vec[index_fopt60]
    print(index_fopt60*500*0.001,pvals_f[index_fopt60])
    #print(index_fopt*500*0.001,min(pvals_f))
    
    table[0][table_index] = varname
    table[1][table_index] = index_fopt60*500*0.001
    table[2][table_index] = pvals_f[index_fopt60]
    
    ###################### 120 hz #####################
    pvals_i = []
    pvals_f = []
    for i in range(len(s_ivec)):
        pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
        pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])
    
    index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))
    
    selected_i_f_normed_dx120 = s_ivec[index_fopt120]
    selected_f_normed_dx120 = s_vec[index_fopt120]
    print(index_fopt120*500*0.001,pvals_f[index_fopt120])
    #print(index_fopt*500*0.001,min(pvals_f))
    
    
    table[3][table_index] = index_fopt120*500*0.001
    table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
from decimal import Decimal
for i in range(len(table[2])):
    table[1][i] = '%.1f' % Decimal(table[1][i])
    table[2][i] = '%.2E' % Decimal(table[2][i])
    table[3][i] = '%.1f' % Decimal(table[3][i])
    table[4][i] = '%.2E' % Decimal(table[4][i])


In [ ]:
table[2]

In [ ]:
save_table = 1

if save_table:
    df = pd.DataFrame(np.swapaxes(table[:-1],0,1), columns=['Variable name','Frequency ($\mathcal{P}_{>60})$','MWU pval  ($\mathcal{P}_{>60}$)', 'Frequency ($\mathcal{P}_{>120})$','MWU pval  ($\mathcal{P}_{>120}$)'])
    df.to_csv('table_pvals.csv')
    

In [ ]:
df

In [ ]:
print(df.to_latex(index=False))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
ytarget = group10.astype(int)

temp_xvariables60 = []
temp_xvariables60.append([selected_i_emd_IF_dx60,selected_i_emd_IA_dx60,selected_f_emd_IF_dx60,selected_f_emd_IA_dx60])
temp_xvariables60.append([selected_i_emd_IF_dy60,selected_i_emd_IA_dy60,selected_f_emd_IF_dy60,selected_f_emd_IA_dy60])
temp_xvariables60.append([selected_i_f_normed_dy60,selected_f_normed_dy60,selected_i_f_normed_dx60,selected_f_normed_dx60])
temp_xvariables60.append([selected_i_f_fix_dy60,selected_f_fix_dy60,selected_i_f_fix_dx60,selected_f_fix_dx60])
temp_xvariables260 = []
for i in range(len(temp_xvariables60)):
    for j in range(len(temp_xvariables60[i])):
        temp_xvariables260.append(temp_xvariables60[i][j])

clf = DecisionTreeClassifier()
xvariables60 = np.swapaxes(temp_xvariables260,0,1)


#cv_scores = cross_val_score(clf, X_selected, ytarget, cv=5)
#print(np.mean(cv_scores))

In [ ]:
vars_list = []
for kk in range(len(dX)):
    vunit_x = np.cos(np.array(angle_single_vector(np.diff(XX[kk][::skip_lag]),np.diff(YY[kk][::skip_lag])))*math.pi/180)
    vunit_y = np.sin(np.array(angle_single_vector(np.diff(XX[kk][::skip_lag]),np.diff(YY[kk][::skip_lag])))*math.pi/180)
    vfix_x = dX[kk][Ssaccade[kk]==1]
    vfix_y = dY[kk][Ssaccade[kk]==1]
    vars_list.append([vunit_x,vunit_y,vfix_x,vfix_y])

In [ ]:


for iii in range(3):
    l_imf = iii
    table_index = 13+iii
    varname = 'IMF'+str(iii+1)+' IF(v_{y,unit})'
    
    i_f_emd_IF_dy = []
    f_emd_IF_dy = []
    i_f_emd_IA_dy = []
    f_emd_IA_dy = []
    
    for k in range(len(dX)):
        if k%10==0:
            print(k)
        vunit = np.sin(np.array(angle_single_vector(np.diff(XX[k][::skip_lag]),np.diff(YY[k][::skip_lag])))*math.pi/180)
        imf = emd.sift.mask_sift(vunit, max_imfs=4)
        IP, IF, IA = emd.spectra.frequency_transform(imf, 1000, 'nht')
        
        cummulative_IF = np.cumsum(ghist)/np.sum(ghist)
        cummulative_IA = np.cumsum(ghist)/np.sum(ghist)
    
        i_f_emd_IF_dy.append([])
        f_emd_IF_dy.append([])
        i_f_emd_IA_dy.append([])
        f_emd_IA_dy.append([])
        for fff in np.arange(0,1,0.001):
            i_f_emd_IF_dy[-1].append(gbins[np.where(cummulative_IF>=fff)[0][0]])
            f_emd_IF_dy[-1].append(cummulative_IF[int(len(cummulative_IF)*fff)])
            i_f_emd_IA_dy[-1].append(gbins[np.where(cummulative_IA>=fff)[0][0]])
            f_emd_IA_dy[-1].append(cummulative_IA[int(len(cummulative_IA)*fff)])
    
    
    ################################
    vec = f_emd_IF_dy
    ivec = i_f_emd_IF_dy
    
    vec = np.array(vec)
    ivec = np.array(ivec)
    s_ivec = np.swapaxes(ivec,0,1)
    s_vec = np.swapaxes(vec,0,1)
    
    
    pvals_i = []
    pvals_f = []
    for i in range(len(s_ivec)):
        pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group10])[1])
        pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group10])[1])
    
    index_fopt60 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group10],axis=0)))
    
    selected_i_f_normed_dx60 = s_ivec[index_fopt60]
    selected_f_normed_dx60 = s_vec[index_fopt60]

    ###################### 120 hz #####################
    pvals_i = []
    pvals_f = []
    for i in range(len(s_ivec)):
        pvals_i.append(scipy.stats.mannwhitneyu(s_ivec[i][group00],s_ivec[i][group11])[1])
        pvals_f.append(scipy.stats.mannwhitneyu(s_vec[i][group00],s_vec[i][group11])[1])
    
    index_fopt120 = np.argmax(np.abs(np.mean(vec[group00],axis=0)-np.mean(vec[group11],axis=0)))
    
    selected_i_f_normed_dx120 = s_ivec[index_fopt120]
    selected_f_normed_dx120 = s_vec[index_fopt120]
    print(index_fopt120*500*0.001,pvals_f[index_fopt120])
    #print(index_fopt*500*0.001,min(pvals_f))
    
    
    table[3][table_index] = index_fopt120*500*0.001
    table[4][table_index] = pvals_f[index_fopt120]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:

#Define the parameter grid
param_grid = {
    'max_depth': [None,2,4,6,8,10,15, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1,2, 5, 10],
    'max_features': [None,1,2,3, 'auto', 'sqrt', 'log2']
}

#param_grid = {
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'max_depth': [None, 10, 20, 30],
#    'min_samples_split': [2, 10, 20],
#    'min_samples_leaf': [1, 5, 10],
#    'max_features': [None, 'auto', 'sqrt', 'log2']
#}
scores60 = []
#Generate all feature combinations
n_features = xvariables60.shape[1]
combinations60 = []
n_combs = 6
for r in range(1, n_combs+1):
    combinations60.extend(itertools.combinations(range(n_features), r))

#Evaluate each combination
clf = DecisionTreeClassifier()
best_score = 0
best_combination = None

for combination in combinations60:
    X_subset = xvariables[:, combination]
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_subset, ytarget)
    mean_cv_score = np.mean(grid_search.best_score_)
    scores60.append(mean_cv_score)
    
    if mean_cv_score > best_score:
        best_score = mean_cv_score
        best_combination = combination
    print(combination)
    print(mean_cv_score)

# Step 5: Select the best combination
print(best_combination)
#print("Best feature combination:", [feature_names[i] for i in best_combination])
print("Best cross-validation score:", best_score)


In [ ]:
ytarget = group11.astype(int)

temp_xvariables120 = []
temp_xvariables120.append([selected_i_emd_IF_dx120,selected_i_emd_IA_dx120,selected_f_emd_IF_dx120,selected_f_emd_IA_dx120])
temp_xvariables120.append([selected_i_emd_IF_dy120,selected_i_emd_IA_dy120,selected_f_emd_IF_dy120,selected_f_emd_IA_dy120])
temp_xvariables120.append([selected_i_f_normed_dy120,selected_f_normed_dy120,selected_i_f_normed_dx120,selected_f_normed_dx120])
temp_xvariables120.append([selected_i_f_fix_dy120,selected_f_fix_dy120,selected_i_f_fix_dx120,selected_f_fix_dx120])
temp_xvariables2120 = []
for i in range(len(temp_xvariables120)):
    for j in range(len(temp_xvariables120[i])):
        temp_xvariables2120.append(temp_xvariables120[i][j])

clf = DecisionTreeClassifier()
xvariables120 = np.swapaxes(temp_xvariables2120,0,1)


#cv_scores = cross_val_score(clf, X_selected, ytarget, cv=5)
#print(np.mean(cv_scores))

In [ ]:
#Define the parameter grid
param_grid = {
    'max_depth': [None,2,4,6,8,10,15, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1,2, 5, 10],
    'max_features': [None,1,2,3, 'auto', 'sqrt', 'log2']
}

#param_grid = {
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'max_depth': [None, 10, 20, 30],
#    'min_samples_split': [2, 10, 20],
#    'min_samples_leaf': [1, 5, 10],
#    'max_features': [None, 'auto', 'sqrt', 'log2']
#}
scores120 = []
#Generate all feature combinations
n_features = xvariables120.shape[1]
combinations120 = []
n_combs = 6
for r in range(1, n_combs+1):
    combinations120.extend(itertools.combinations(range(n_features), r))

#Evaluate each combination
clf = DecisionTreeClassifier()
best_score = 0
best_combination = None

for combination in combinations120:
    X_subset = xvariables[:, combination]
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_subset, ytarget)
    mean_cv_score = np.mean(grid_search.best_score_)
    scores120.append(mean_cv_score)
    
    if mean_cv_score > best_score:
        best_score = mean_cv_score
        best_combination = combination
    print(combination)
    print(mean_cv_score)

# Step 5: Select the best combination
print(best_combination)
#print("Best feature combination:", [feature_names[i] for i in best_combination])
print("Best cross-validation score:", best_score)

In [ ]:
best_score

In [ ]:
np.sum(np.array(scores)>0.9)

In [ ]:
np.logspace(-1,3,10)

In [ ]:
param_grid = {
    'C': np.logspace(-1,3,10),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'shrinking': [True, False]
}
param_grid = {
    'C': [0.1,1,5,10,25,50,100],
    'kernel': ['linear', 'sigmoid']
}
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

#Generate all feature combinations
n_features = xvariables.shape[1]
combinations = []
n_combs = 3
for r in range(1, n_combs+1):
    combinations.extend(itertools.combinations(range(n_features), r))

#Evaluate each combination
clf = SVC()
best_score = 0
best_combination = None

for combination in combinations:
    X_subset = xvariables[:, combination]
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_subset, ytarget)
    mean_cv_score = np.mean(grid_search.best_score_)
    
    if mean_cv_score > best_score:
        best_score = mean_cv_score
        best_combination = combination
    print(combination)
    print(mean_cv_score)

# Step 5: Select the best combination
print(best_combination)
#print("Best feature combination:", [feature_names[i] for i in best_combination])
print("Best cross-validation score:", best_score)


In [ ]:


# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 3,4,5,'sqrt', 'log2']
}
param_grid = {
    'n_estimators': [100]
}

#Generate all feature combinations
n_features = xvariables.shape[1]
combinations = []
n_combs = 4
for r in range(3, n_combs+1):
    combinations.extend(itertools.combinations(range(n_features), r))


#Evaluate each combination
clf = RandomForestClassifier(random_state=0)
best_score = 0
best_combination = None

for combination in combinations:
    X_subset = xvariables[:, combination]
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_subset, ytarget)
    mean_cv_score = np.mean(grid_search.best_score_)
    
    if mean_cv_score > best_score:
        best_score = mean_cv_score
        best_combination = combination
    print(combination)
    print(mean_cv_score)

# Step 5: Select the best combination
print(best_combination)
#print("Best feature combination:", [feature_names[i] for i in best_combination])
print("Best cross-validation score:", best_score)


In [ ]:
best_score

In [ ]:
best_combination

In [ ]:
pvals_IF_f

In [ ]:
s_f_emd_IF_dx

In [ ]:
IA[:, 2]

In [ ]:
pvals_peak = []
pvals_60 = []
pvals_60_plus = []

print(scipy.stats.mannwhitneyu(peak_IF[sepa:],peak_IF[:sepa])[1])
print(scipy.stats.mannwhitneyu(peak_IF60[sepa:],peak_IF60[:sepa])[1])
print(scipy.stats.mannwhitneyu(peak_IF60plus[sepa:],peak_IF60plus[:sepa])[1])
print(scipy.stats.mannwhitneyu(peak_IA[sepa:],peak_IA[:sepa])[1])
print(scipy.stats.mannwhitneyu(peak_IA60[sepa:],peak_IA60[:sepa])[1])
print(scipy.stats.mannwhitneyu(peak_IA60plus[sepa:],peak_IA60plus[:sepa])[1])



In [ ]:
plt.hist(peak_IF60[sepa:],np.arange(0.005,0.02,0.0005),density = True)
plt.hist(peak_IF60[:sepa],np.arange(0.005,0.02,0.0005),density = True,alpha=0.5)

In [ ]:
plt.hist(peak_IA60plus[sepa:],np.arange(0.05,0.2,0.005),density = True)
plt.hist(peak_IA60plus[:sepa],np.arange(0.05,0.2,0.005),density = True,alpha=0.5)

In [ ]:
plt.hist(peak_IA60plus[sepa:],density = True)
plt.hist(peak_IA60plus[:sepa],density = True,alpha=0.5)

In [ ]:
c =  plt.hist(IF[:, 2], np.linspace(0, 150))

In [ ]:
a[1][np.argmax(a[0])]

In [ ]:
a

In [ ]:
np.linspace(0, 150)[np.argmax(IF[:, 2])]

In [ ]:
np.argmax(IF[:, 2])

In [ ]:
f, spectrum = emd.spectra.hilberthuang(IF, IA, (0, 100, 25))

freq_edges, freq_centres = emd.spectra.define_hist_bins(0, 100, 128, 'linear')

# Amplitude weighted HHT per IMF
f, spec_weighted = emd.spectra.hilberthuang(IF, IA, freq_edges, sum_imfs=False)

# Unweighted HHT per IMF - we replace the instantaneous amplitude values with ones
f, spec_unweighted = emd.spectra.hilberthuang(IF, np.ones_like(IA), freq_edges, sum_imfs=False)

plt.figure(figsize=(10, 4))
plt.subplots_adjust(hspace=0.4)
plt.subplot(121)
plt.plot(freq_centres, spec_unweighted)
plt.xticks(np.arange(10)*10)
plt.xlim(0, 100)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Count')
plt.title('unweighted\nHilbert-Huang Transform')

plt.subplot(122)
plt.plot(freq_centres, spec_weighted)
plt.xticks(np.arange(10)*10)
plt.xlim(0, 100)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.title('IA-weighted\nHilbert-Huang Transform')
plt.legend(['IMF-1', 'IMF-2', 'IMF-3', 'IMF-4', 'IMF-5'], frameon=False)

In [ ]:
IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'nht')

In [ ]:
frequencies[np.where(np.cumsum(moving_averages)/np.sum(moving_averages)>0.8)[0][0]]

In [ ]:
Energy = []
N = 500
skip_lag = 3
frequency = 1000/skip_lag

for k in range(len(dX)):
    norm_dx = np.cos(np.array(angle_single_vector(np.diff(XX[k][::skip_lag]),np.diff(YY[k][::skip_lag])))*math.pi/180)
    l_P_S = power_spectrum(norm_dx,frequency,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))
    #plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    #plt.show()
    
    Energy.append(np.sum(l_P_S))

    norm_dy = np.sin(np.array(angle_single_vector(np.diff(XX[k][::skip_lag]),np.diff(YY[k][::skip_lag])))*math.pi/180)
    l_P_S = power_spectrum(norm_dy,frequency,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))
    #plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))

In [ ]:
Energy = []
N = 1000
skip_lag = 4
frequency = 1000/skip_lag

for k in range(len(dX)):
    norm_dx = np.cos(np.array(angle_single_vector(np.diff(XX[k][int(len(XX[k])/2)::skip_lag]),np.diff(YY[k][int(len(YY[k])/2)::skip_lag])))*math.pi/180)
    l_P_S = power_spectrum(norm_dx,frequency,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))
    #plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")

    
    Energy.append(np.sum(l_P_S))


    norm_dx = np.cos(np.array(angle_single_vector(np.diff(XX[k][:int(len(XX[k])/2):skip_lag]),np.diff(YY[k][:int(len(XX[k])/2):skip_lag])))*math.pi/180)
    l_P_S = power_spectrum(norm_dx,frequency,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))
    #plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")

    #plt.vlines(15, min(np.log(moving_averages)), max(np.log(moving_averages)), colors='k', linestyles='solid')
    #plt.vlines(30, min(np.log(moving_averages)), max(np.log(moving_averages)), colors='k', linestyles='solid')
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))

In [ ]:
Energy = []
N = 100
frequency = 1000
for k in range(len(dX)):
    l_P_S = power_spectrum(dX[k],1000,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,moving_averages)

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))
      

    
    

In [ ]:
Energy = []
N = 100
frequency = 1000
for k in range(len(dX)):
    l_P_S = power_spectrum(dX[k],1000,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))

In [ ]:
Energy = []
N = 100
frequency = 1000
for k in range(len(dY)):
    l_P_S = power_spectrum(dY[k],1000,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies,np.log(moving_averages))

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))

In [ ]:
Energy = []
N = 100
frequency = 1000
for k in range(len(dX)):
    l_P_S = power_spectrum(dX[k],1000,False,'')
    # Program to calculate moving average

    frequencies = np.linspace(0, frequency/2, len(l_P_S)-N+1)
    
    moving_averages =  np.convolve(l_P_S, np.ones(N)/N, mode='valid')

        
    plt.plot(frequencies[:int(len(frequencies)/10)],moving_averages[:int(len(frequencies)/10)])

    plt.title("Power Spectrum")
    plt.xlabel("frequency (HZ)")
    
    plt.show()
    
    Energy.append(np.sum(l_P_S))
      

    

In [ ]:
plt.plot(np.log2(Energy))

In [ ]:
import scipy.fftpack
from mpl_toolkits.mplot3d import Axes3D
def centers(edges):
        return( edges[:-1] + np.diff(edges[:2])/2)
def fft_2d(lvector1, lvector2, nbins,graph):
    
    histogram, xedges, yedges = np.histogram2d(lvector1, lvector2, bins=nbins)
    # Figure out centers of bins
    x_centers = centers(xedges)
    y_centers = centers(yedges)
    
    #"""CREATING REAL AND MOMENTUM SPACES GRIDS"""
    N_x,N_y = len(histogram),len(histogram)
    range_x, range_y = np.arange(N_x), np.arange(N_y)
    
    dk_x, dk_y = np.pi / np.max(x_centers), np.pi / np.max(y_centers)
# momentum space grid vectors, shifted to center for zero frequency
    k_xv, k_yv = dk_x * np.append(range_x[:N_x//2], -range_x[N_x//2:0:-1]), \
             dk_y * np.append(range_y[:N_y//2], -range_y[N_y//2:0:-1])
# create real and momentum spaces grids
    x, y = np.meshgrid(x_centers, y_centers, sparse=False, indexing='ij')
    kx, ky = np.meshgrid(k_xv, k_yv, sparse=False, indexing='ij')
    F = scipy.fftpack.fft2(histogram)
    if graph:
        fig = plt.figure()
        ax = Axes3D(fig)
        surf = ax.plot_surface(x, y, np.abs(histogram), cmap='viridis')
        plt.show()
        
        fig = plt.figure()
        ax = Axes3D(fig)
        surf = ax.plot_surface(x, y, np.log10(np.abs(histogram)), cmap='viridis')
        plt.show()
         # for other plots I changed to
#        fig2 = plt.figure()
#        ax2 =Axes3D(fig2)
#        surf = ax2.plot_surface(kx, ky, np.abs(F)*dx*dy, cmap='viridis')
#        plt.show()
    


        

In [ ]:
fft_2d(dX[0],dY[0],30,True)

In [ ]:
for k in range(len(dX)):
    corr_OG = []
    temp_ldx = np.diff(X[k][saccade[k]==1])
    for i in range(1,50):
        #corr_OG.append( np.corrcoef(np.hstack(d_X_num_clean_lag[0][7])[i:],np.hstack(d_X_num_clean_lag[0][7])[:-i])[1,0])
        corr_OG.append( np.corrcoef(temp_ldx[i:],temp_ldx[:-i])[1,0])

    plt.plot(np.arange(1,50),corr_OG)
    plt.ylim(1.1*min(-0.15,min(corr_OG)),1.1*max(0.15,max(corr_OG)))
    plt.hlines([0],1,50,linestyles='--',color='k') 
    plt.xticks(np.arange(1,50,5))
    plt.title('Eye-tracker data: horizontal displacements')
    plt.xlabel('lag')
    plt.ylabel('correlation coefficient')
    plt.show()

In [ ]:

        
############################# #1 - one variable ########################



def frequency_matrix_2D(d__ss,threshold,normalized):
    d__ss = np.array(d__ss)
    d__ss = (d__ss-min(d__ss))/((max(d__ss)-min(d__ss))*1.000001)
    binary_vector = np.array(d__ss>threshold).astype(int)
    matrix = np.histogram2d(binary_vector[1:], binary_vector[:-1],[0,1,2])[0]
    if normalized:
        for j in range(2):
            matrix[j,:] = matrix[j,:]/matrix.sum(axis=1)[j]
    return( matrix)
    

        
                    
    
        

#in threshold array, 0 corresponds to the minimum of the vector while 1 is the maximum of the vector
def form_groups(vector,threshold_array,graph,x_label,title,x_axis_format):
    
    detectionfisher=[]
    detection=[]
    for i in threshold_array:
        matrix = frequency_matrix_2D(vector,i,False)
        detectionfisher.append(np.log(scipy.stats.fisher_exact(matrix)[1]))
        p = matrix.sum(axis=1)[0]/matrix.sum()
        if p==1 or p==0:
            detection.append(1)
        else:
            detection.append((matrix[1][0]) / (matrix.sum() *(p*(1-p))))

    
    minim = min(vector)
    diff = max(vector) - minim
    min_k = np.argmin(detection)*diff + minim
    min_fisher = np.argmin(detectionfisher)*diff + minim
    
    if graph:

        xticks_labels = [x_axis_format % (minim + diff*pipi) for pipi in threshold_array]
        
        
        plt.plot(detection)
        plt.xlabel(x_label)
        #plt.title(title +"k min = " +str(round(min_k,5)))
        plt.title(title)
        if len(threshold_array) > 40:
            plt.xticks(np.arange(len(threshold_array))[::int(len(threshold_array)/10)],xticks_labels[::int(len(threshold_array)/10)])
        else:
            plt.xticks(np.arange(len(threshold_array))[::4],xticks_labels[::4])
        plt.ylabel("k")
        plt.savefig("group_detection - k " + x_label + title+".png",dpi = 500)
        plt.show()
        plt.close()

        
        plt.plot(detectionfisher)
        plt.xlabel(x_label)
        plt.title(title)
        if len(threshold_array) > 40:
            plt.xticks(np.arange(len(threshold_array))[::int(len(threshold_array)/10)],xticks_labels[::int(len(threshold_array)/10)])
        else:
            plt.xticks(np.arange(len(threshold_array))[::4],xticks_labels[::4])
        plt.ylabel("log-fisher exact test")
        plt.savefig("group_detection - log-fisher " + x_label +title+ ".png",dpi = 500)
        plt.show()
        plt.close()
    
   
    return(detection, detectionfisher, min_k,min_fisher)




def real_k_and_fisher(binary_vector):
    matrix = np.zeros((2,2))
    for i in range(len(binary_vector)-1):
        matrix[int(binary_vector[i])][int(binary_vector[i+1])] +=1
    
    detectionfisher=[]
    detection=[]

    detectionfisher.append(np.log(scipy.stats.fisher_exact(matrix)[1]))
    p = matrix.sum(axis=1)[0]/matrix.sum()
    detection.append((matrix[1][0])/(matrix.sum()*(p*(1-p))))

    return( matrix,detection, detectionfisher)

In [ ]:
vS_tot = np.hstack(dS) / np.hstack(dT)

In [ ]:
ldetection, ldetectionfisher, lkmin, lfishermin = form_groups(vS_tot,np.arange(1,25,0.25)/max(vS_tot),True,'v','title','%.2f')


In [ ]:
vS_tot = np.hstack(dS) / np.hstack(dT)

In [ ]:
detection_list = []
kmin_list = []
for i in range(len(dS)):
    #lc_vS = np.array(dS[i]) / np.array(dT[i])
    lc_vS = np.array(dS[i]) 
    raw_threshold_array = np.arange(1,30,0.5)
    threshold_array = raw_threshold_array / max(lc_vS)
    ldetection, ldetectionfisher, lkmin, lfishermin = form_groups(lc_vS,threshold_array,True,'v','title','%.2f')
    detection_list.append(ldetection)
    kmin_list.append(lkmin)
    print(i)

In [ ]:
import scipy.signal
N=5
for i in range(len(detection_list)):
    print(i)
    lc_smooth_det = np.convolve(detection_list[i], np.ones(N), 'valid') / N
    plt.plot(lc_smooth_det)
    minima = scipy.signal.argrelextrema(lc_smooth_det, np.less)
    plt.vlines(minima,0,0.3,color='k')
    print(scipy.signal.argrelextrema(lc_smooth_det, np.less))
    plt.show()
    #plt.plot(np.diff(lc_smooth_det))
    #plt.vlines(minima,0,0.3,color='k')
    #plt.show()
    #plt.plot(np.diff(np.diff(lc_smooth_det)))
    #plt.vlines(minima,0,0.3,color='k')
    #plt.show()
    
included = [0,2,4,5,6,7,8,9,10,11,12,13,14,16,18,19,20]    
excluded = [1,3,15,17,21,22]    
    

In [ ]:
import scipy.signal
N=5
for i in range(len(detection_list)):
    print(i)
    lc_smooth_det = np.convolve(detection_list[i], np.ones(N), 'valid') / N
    plt.plot(lc_smooth_det)
    minima = scipy.signal.argrelextrema(lc_smooth_det, np.less)
    plt.vlines(minima,0,0.3,color='k')
    print(scipy.signal.argrelextrema(lc_smooth_det, np.less))
    plt.show()
    #plt.plot(np.diff(lc_smooth_det))
    #plt.vlines(minima,0,0.3,color='k')
    #plt.show()
    #plt.plot(np.diff(np.diff(lc_smooth_det)))
    #plt.vlines(minima,0,0.3,color='k')
    #plt.show()
    
included = [0,2,4,5,6,7,8,9,10,11,12,13,14,16,18,19,20]    
excluded = [1,3,15,17,21,22]  

In [ ]:
import scipy.signal
N=5
included = [0,2,4,5,6,7,8,9,10,11,12,13,14,16,18,19,20]    
excluded = [1,3,15,17,21,22]  
min_first_min = 10 #first minimum should be lower than this number.
gminima =[]
for i in range(len(detection_list)):
    if i in included:
        print(i)
        lc_smooth_det = np.convolve(detection_list[i], np.ones(N), 'valid') / N
        temp_minima = scipy.signal.argrelextrema(lc_smooth_det, np.less)[0]
        gminima.append([])
        if temp_minima[0]>min_first_min:
            gminima[-1].append(0)
            gminima[-1].append(temp_minima[0])
        else: 
            gminima[-1].append(temp_minima[0])
            if temp_minima[1] <10:
                gminima[-1].append(temp_minima[2])
            else:
                gminima[-1].append(temp_minima[1])
            
        plt.plot(lc_smooth_det)
        plt.vlines(gminima[-1],0,0.3,color='k')
        print(scipy.signal.argrelextrema(lc_smooth_det, np.less))
        #plt.savefig()
        plt.show()


In [ ]:
########## 2 types of movement ##############

In [ ]:
#

est_sacc_mu = []
est_sacc_sigma = []
est_fix_mu = []
est_fix_sigma = []
est_lambda_B_list = [] #transition saccade to fix 
est_lambda_D_list = [] # vice versa
guess_logdS_fix_list = []
guess_logdS_sacc_list = []
vS = []
for i in range(len(included)):
    vS.append(np.array(dS[included[i]]) / np.array(dT[included[i]]))
for i in range(len(included)):
    lthreshold = raw_threshold_array[gminima[i][0]]
    d__ss = vS[i]
    binary_vector = np.array([max(min(int(i-lthreshold+1),1),0) for i in d__ss])
    guess_dS_fix = vS[i][binary_vector==0]
    guess_dS_sacc = vS[i][binary_vector==1]
    guess_logdS_fix = np.log(guess_dS_fix[guess_dS_fix>0])
    guess_logdS_fix_list.append(guess_logdS_fix)
    guess_logdS_sacc = np.log(guess_dS_sacc[guess_dS_sacc>0])
    guess_logdS_sacc_list.append(guess_logdS_sacc)
    est_fix_mu.append(np.mean(guess_logdS_fix))
    est_fix_sigma.append(np.std(guess_logdS_fix))
    est_sacc_mu.append(np.mean(guess_logdS_sacc))
    est_sacc_sigma.append(np.std(guess_logdS_sacc))
    Nfix = len(binary_vector) - np.sum(binary_vector)
    Nsacc = np.sum(binary_vector)
    Ntransi = int(np.sum(np.abs(binary_vector[1:] - binary_vector[:-1]))/2)
    est_lambda_B_list.append(-np.log(Ntransi / Nsacc))
    est_lambda_D_list.append(-np.log(Ntransi / Nfix))

In [ ]:
gminima

In [ ]:
from pomegranate import *
HMM_est_sacc_mu = []
HMM_est_sacc_sigma = []
HMM_est_fix_mu = []
HMM_est_fix_sigma = []
HMM_est_D_list = []
HMM_est_lambda_B_list = []
HMM_est_lambda_D_list = []
HMM_logdS_fix_list = []
HMM_logdS_sacc_list = []


for i in range(len(included)-1):
    print(i)
    prbd = np.exp(-est_lambda_B_list[i])
    prbb = 1-prbd
    prdb = np.exp(-est_lambda_D_list[i])
    prdd = 1-prdb
    s1 = State(LogNormalDistribution(est_fix_mu[i], est_fix_sigma[i])) #fixations
    s2 = State(LogNormalDistribution(est_sacc_mu[i], est_sacc_sigma[i])) #saccades
    model = HiddenMarkovModel()
    model.add_states(s1, s2)
    model.add_transition(model.start, s1, 0.5)
    model.add_transition(model.start, s2, 0.5)
    model.add_transition(s1, s1, prdd)
    model.add_transition(s1, s2, prdb)
    model.add_transition(s2, s1, prbd)
    model.add_transition(s2, s2, prbb)
    model.add_transition(s2, model.end, 0.5*len(vS[i][vS[i]>0]))
    model.add_transition(s1, model.end, 0.5*len(vS[i][vS[i]>0]))
    model.bake()
    model.fit( [vS[i][vS[i]>0]] )
    temp_vel = vS[i][vS[i]>0]
    
    s1_mu = model.states[0].distribution.parameters[0]
    s2_mu = model.states[1].distribution.parameters[0]
    
    if s2_mu > s1_mu:
        HMM_est_fix_mu.append(model.states[0].distribution.parameters[0])
        HMM_est_fix_sigma.append(model.states[0].distribution.parameters[1])
        HMM_est_sacc_mu.append(model.states[1].distribution.parameters[0])
        HMM_est_sacc_sigma.append(model.states[1].distribution.parameters[1])
        HMM_bin_vec = np.swapaxes(model.viterbi(vS[i][vS[i]>0])[1],0,1)[0][1:-1]

        
        
        
    else:
        HMM_est_fix_mu.append(model.states[1].distribution.parameters[0])
        HMM_est_fix_sigma.append(model.states[1].distribution.parameters[1])
        HMM_est_sacc_mu.append(model.states[0].distribution.parameters[0])
        HMM_est_sacc_sigma.append(model.states[0].distribution.parameters[1])
        HMM_bin_vec = np.swapaxes(model.viterbi(vS[i][vS[i]>0])[1],0,1)[0][1:-1]
        HMM_bin_vec = np.abs(np.array(HMM_bin_vec) - np.ones(len(HMM_bin_vec)))

    Nfix = len(HMM_bin_vec) - np.sum(HMM_bin_vec)
    Nsacc = np.sum(HMM_bin_vec)
    Ntransi = int(np.sum(np.abs(HMM_bin_vec[1:] - HMM_bin_vec[:-1]))/2)
    HMM_est_lambda_B_list.append(-np.log(1 - Ntransi / Nsacc))
    HMM_est_lambda_D_list.append(-np.log(1 - Ntransi / Nfix))
    HMM_logdS_fix_list.append(temp_vel[HMM_bin_vec==0])
    HMM_logdS_sacc_list.append(temp_vel[HMM_bin_vec==1])

    

In [ ]:
for i in range(len(included)):
    plt.hist(np.log(HMM_logdS_fix_list[i]),np.arange(-2,4,0.2),density=False);
    plt.hist(np.log(HMM_logdS_sacc_list[i]),np.arange(-2,4,0.2),density=False,alpha=0.5);
    plt.show()

In [ ]:
for i in range(len(included)):
    plt.hist(np.log(vS[included[i]][vS[included[i]]>0] ),np.arange(-2,3,0.2),density=True);
    plt.show()

In [ ]:
############# 3 states

In [ ]:
included = [0,2,4,5,6,7,8,9,10,11,12,13,14,16,18,19,20]  

In [ ]:

est_pliss_mu = []
est_pliss_sigma = []
est_sacc_mu = []
est_sacc_sigma = []
est_fix_mu = []
est_fix_sigma = []

guess_logdS_fix_list = []
guess_logdS_sacc_list = []
guess_logdS_pliss_list = []
est_lambda_DB_list= []
est_lambda_DP_list= []
est_lambda_BD_list= []
est_lambda_BP_list= []
est_lambda_PD_list= []
est_lambda_PB_list= []
bin_vec_vec = []
for i in range(len(gminima)):
    lthreshold = raw_threshold_array[gminima[i][0]]
    lthreshold2 = raw_threshold_array[gminima[i][1]]
    d__ss = vS[i]
    binary_vector1 = np.array([max(min(int(i-lthreshold+1),1),0) for i in d__ss])
    binary_vector2 = np.array([max(min(int(i-lthreshold2+1),1),0) for i in d__ss])
    binary_vector = binary_vector1 + binary_vector2
    guess_dS_fix = vS[i][binary_vector==0]
    guess_dS_sacc = vS[i][binary_vector==1]
    guess_dS_pliss = vS[i][binary_vector==2]
    
    guess_logdS_fix = np.log(guess_dS_fix[guess_dS_fix>0])
    guess_logdS_fix_list.append(guess_logdS_fix)
    guess_logdS_sacc = np.log(guess_dS_sacc[guess_dS_sacc>0])
    guess_logdS_sacc_list.append(guess_logdS_sacc)
    guess_logdS_pliss = np.log(guess_dS_pliss[guess_dS_pliss>0])
    guess_logdS_pliss_list.append(guess_dS_pliss)
    
    est_fix_mu.append(np.mean(guess_logdS_fix))
    est_fix_sigma.append(np.std(guess_logdS_fix))
    est_sacc_mu.append(np.mean(guess_logdS_sacc))
    est_sacc_sigma.append(np.std(guess_logdS_sacc))
    est_pliss_mu.append(np.mean(guess_logdS_pliss))
    est_pliss_sigma.append(np.std(guess_logdS_pliss))
    bin_vec_vec.append(binary_vector)
    
    bin_vec_sq = binary_vector**2
    diff_bin_vec_sq = binary_vector[1:]**2 - binary_vector[:-1]**2
    
    
    Nfix = np.sum(binary_vector==0)
    Nsacc = np.sum(binary_vector==1)
    Npliss = np.sum(binary_vector==2)
    
    Nfixsacc =  np.sum(diff_bin_vec_sq==1)
    Nfixpliss = np.sum(diff_bin_vec_sq==4)
    Nsaccfix = np.sum(diff_bin_vec_sq==-1)
    Nsaccpliss = np.sum(diff_bin_vec_sq==3)
    Nplissfix = np.sum(diff_bin_vec_sq==-4)
    Nplisssacc= np.sum(diff_bin_vec_sq==-3)
    
    Nfixfix = Nfix - Nfixsacc - Nfixpliss
    Nsaccsacc = Nsacc - Nsaccfix -Nsaccpliss
    Nplisspliss = Npliss - Nplissfix - Nplisssacc
    

    
    
    est_lambda_DB_list.append(-np.log((Nfixsacc +1) / Nfix))
    est_lambda_DP_list.append(-np.log((Nfixpliss+1) / Nfix))
    est_lambda_BD_list.append(-np.log((Nsaccfix +1)/ Nsacc))
    est_lambda_BP_list.append(-np.log((Nsaccpliss+1) / Nsacc))
    est_lambda_PD_list.append(-np.log((Nplissfix +1)/ Npliss))
    est_lambda_PB_list.append(-np.log((Nplisssacc +1)/ Npliss))
    

    
    
    

In [ ]:
gminima

In [ ]:
for i in range(len(included)):
    plt.hist(np.log(vS[i][bin_vec_vec[i]==0]+0.1),np.arange(-2,5,0.1),density=False);
    plt.hist(np.log(vS[i][bin_vec_vec[i]==1]+0.1),np.arange(-2,5,0.1),density=False,alpha=0.6);
    plt.hist(np.log(vS[i][bin_vec_vec[i]==2]+0.1),np.arange(-2,5,0.1),density=False,alpha=0.6);
    plt.show()

In [ ]:
for i in range(len(included)):
    plt.hist(np.log(vS[i][bin_vec_vec[i]==2]+0.1),np.arange(-2,5,0.1),density=False,alpha=0.6);
    plt.show()

In [ ]:
HMM_est_pliss_mu_b = []
HMM_est_pliss_sigma_b = []
HMM_est_sacc_mu_b = []
HMM_est_sacc_sigma_b = []
HMM_est_fix_mu_b = []
HMM_est_fix_sigma_b = []
HMM_est_D_list_b = []
HMM_est_lambda_SF_list_b = []
HMM_est_lambda_SP_list_b = []
HMM_est_lambda_FS_list_b = []
HMM_est_lambda_FP_list_b = []
HMM_est_lambda_PS_list_b = []
HMM_est_lambda_PF_list_b = []

HMM_logdS_fix_list_b = []
HMM_logdS_sacc_list_b = []
HMM_logdS_pliss_list_b = []

for i in range(len(included)):
    print(i)
    #############Pr Fixations ######################
    prdb = np.exp(-est_lambda_DB_list[i])
    prdp = np.exp(-est_lambda_DP_list[i])
    prdd = 1-prdb - prdp
    #############Pr Saccades########################
    prbd = np.exp(-est_lambda_BD_list[i])
    prbp = np.exp(-est_lambda_BP_list[i])
    prbb = 1-prbd - prbp
    #############Pr plissades ######################
    prpd = np.exp(-est_lambda_PD_list[i])
    prpb = np.exp(-est_lambda_PB_list[i])
    prpp = 1-prpb - prpd
    ################################################


    
    s1 = State(LogNormalDistribution(est_fix_mu[i], est_fix_sigma[i])) #fixations
    s2 = State(LogNormalDistribution(1.3*est_sacc_mu[i], est_sacc_sigma[i])) #saccades
    s3 = State(LogNormalDistribution(1.3*est_pliss_mu[i], est_pliss_sigma[i])) #plissades
    model = HiddenMarkovModel()
    model.add_states(s1, s2, s3)
    model.add_transition(model.start, s1, 0.4)
    model.add_transition(model.start, s2, 0.3)
    model.add_transition(model.start, s3, 0.3)
    model.add_transition(s1, s1, prdd)
    model.add_transition(s1, s2, prdb)
    model.add_transition(s1, s3, prdp)
    model.add_transition(s2, s1, prbd)
    model.add_transition(s2, s2, prbb)
    model.add_transition(s2, s3, prbp)
    model.add_transition(s3, s1, prpd)
    model.add_transition(s3, s2, prpb)
    model.add_transition(s3, s3, prpp)
    model.add_transition(s1, model.end, 0.4*len(vS[i][vS[i]>0]))
    model.add_transition(s2, model.end, 0.3*len(vS[i][vS[i]>0]))
    model.add_transition(s3, model.end, 0.3*len(vS[i][vS[i]>0]))
    model.bake()
    model.fit( [vS[i][vS[i]>0]] )
    temp_vel = vS[i][vS[i]>0]
    
    s1_mu = model.states[0].distribution.parameters[0]
    s2_mu = model.states[1].distribution.parameters[0]
    s3_mu = model.states[2].distribution.parameters[0]
    pos_array = np.argsort([s1_mu,s2_mu,s3_mu])
    pos_array2 = np.array([np.where(np.argsort([s1_mu,s2_mu,s3_mu]) ==0)[0][0], np.where(np.argsort([s1_mu,s2_mu,s3_mu]) ==1)[0][0],np.where(np.argsort([s1_mu,s2_mu,s3_mu]) ==2)[0][0]])
 
    HMM_est_fix_mu_b.append(model.states[pos_array[0]].distribution.parameters[0])
    HMM_est_fix_sigma_b.append(model.states[pos_array[0]].distribution.parameters[1])
    HMM_est_sacc_mu_b.append(model.states[pos_array[1]].distribution.parameters[0])
    HMM_est_sacc_sigma_b.append(model.states[pos_array[1]].distribution.parameters[1])
    HMM_est_pliss_mu_b.append(model.states[pos_array[2]].distribution.parameters[0])
    HMM_est_pliss_sigma_b.append(model.states[pos_array[2]].distribution.parameters[1])
    temp_HMM_bin_vec = np.swapaxes(model.viterbi(vS[i][vS[i]>0])[1],0,1)[0][1:-1]
    HMM_bin_vec = np.array([pos_array2[kk] for kk in temp_HMM_bin_vec])

        
        

    #Nfix = len(HMM_bin_vec) - np.sum(HMM_bin_vec)
    #Nsacc = np.sum(HMM_bin_vec)
    #Ntransi = int(np.sum(np.abs(HMM_bin_vec[1:] - HMM_bin_vec[:-1]))/2)
    #HMM_est_lambda_B_list.append(-np.log(1 - Ntransi / Nsacc))
    #HMM_est_lambda_D_list.append(-np.log(1 - Ntransi / Nfix))
    HMM_logdS_fix_list_b.append(temp_vel[HMM_bin_vec==0])
    HMM_logdS_sacc_list_b.append(temp_vel[HMM_bin_vec==1])
    HMM_logdS_pliss_list_b.append(temp_vel[HMM_bin_vec==2])

    

In [ ]:
for i in range(len(included)):
    plt.hist(np.log(HMM_logdS_fix_list_b[i]),np.arange(-2,3,0.2),density=False);
    plt.hist(np.log(HMM_logdS_sacc_list_b[i]),np.arange(-2,3,0.2),density=False,alpha=0.6);
    plt.hist(np.log(HMM_logdS_pliss_list_b[i]),np.arange(-2,3,0.2),density=False,alpha=0.4);
    plt.show()

In [ ]:
ldetection, ldetectionfisher, lkmin, lfishermin = form_groups(vS_tot,np.arange(1,40,1)/max(vS_tot),True,'v','title','%.2f')


In [ ]:
vS_tot = np.abs(np.hstack(vS) +0.05)

In [ ]:
np.log(13)

In [ ]:
plt.hist(np.log(vS_tot),60);
plt.vlines(2.7,0,300000)

In [ ]:
plt.hist(np.log(vS_tot[vS_tot>0]),100);
plt.vlines(2.7,0,500000,color='r')
plt.vlines(0.5,0,500000,color='k')



In [ ]:
plt.hist(np.log(vS_tot[vS_tot>2]),200);
plt.vlines(2.7,0,5000,color='r')

In [ ]:
lvS = [vS_tot]


est_pliss_mu = []
est_pliss_sigma = []
est_sacc_mu = []
est_sacc_sigma = []
est_fix_mu = []
est_fix_sigma = []

guess_logdS_fix_list = []
guess_logdS_sacc_list = []
guess_logdS_pliss_list = []
est_lambda_DB_list= []
est_lambda_DP_list= []
est_lambda_BD_list= []
est_lambda_BP_list= []
est_lambda_PD_list= []
est_lambda_PB_list= []
bin_vec_vec = []
for i in range(len(lvS)):
    lthreshold = 3
    lthreshold2 = 15
    d__ss = lvS[i]
    binary_vector1 = np.array([max(min(int(i-lthreshold+1),1),0) for i in d__ss])
    binary_vector2 = np.array([max(min(int(i-lthreshold2+1),1),0) for i in d__ss])
    binary_vector = binary_vector1 + binary_vector2
    guess_dS_fix = lvS[i][binary_vector==0]
    guess_dS_sacc = lvS[i][binary_vector==1]
    guess_dS_pliss = lvS[i][binary_vector==2]
    
    guess_logdS_fix = np.log(guess_dS_fix[guess_dS_fix>0])
    guess_logdS_fix_list.append(guess_logdS_fix)
    guess_logdS_sacc = np.log(guess_dS_sacc[guess_dS_sacc>0])
    guess_logdS_sacc_list.append(guess_logdS_sacc)
    guess_logdS_pliss = np.log(guess_dS_pliss[guess_dS_pliss>0])
    guess_logdS_pliss_list.append(guess_dS_pliss)
    
    est_fix_mu.append(np.mean(guess_logdS_fix))
    est_fix_sigma.append(np.std(guess_logdS_fix))
    est_sacc_mu.append(np.mean(guess_logdS_sacc))
    est_sacc_sigma.append(np.std(guess_logdS_sacc))
    est_pliss_mu.append(np.mean(guess_logdS_pliss))
    est_pliss_sigma.append(np.std(guess_logdS_pliss))
    bin_vec_vec.append(binary_vector)
    
    bin_vec_sq = binary_vector**2
    diff_bin_vec_sq = binary_vector[1:]**2 - binary_vector[:-1]**2
    
    
    Nfix = np.sum(binary_vector==0)
    Nsacc = np.sum(binary_vector==1)
    Npliss = np.sum(binary_vector==2)
    
    Nfixsacc =  np.sum(diff_bin_vec_sq==1)
    Nfixpliss = np.sum(diff_bin_vec_sq==4)
    Nsaccfix = np.sum(diff_bin_vec_sq==-1)
    Nsaccpliss = np.sum(diff_bin_vec_sq==3)
    Nplissfix = np.sum(diff_bin_vec_sq==-4)
    Nplisssacc= np.sum(diff_bin_vec_sq==-3)
    
    Nfixfix = Nfix - Nfixsacc - Nfixpliss
    Nsaccsacc = Nsacc - Nsaccfix -Nsaccpliss
    Nplisspliss = Npliss - Nplissfix - Nplisssacc
    

    
    
    est_lambda_DB_list.append(-np.log((Nfixsacc +1) / Nfix))
    est_lambda_DP_list.append(-np.log((Nfixpliss+1) / Nfix))
    est_lambda_BD_list.append(-np.log((Nsaccfix +1)/ Nsacc))
    est_lambda_BP_list.append(-np.log((Nsaccpliss+1) / Nsacc))
    est_lambda_PD_list.append(-np.log((Nplissfix +1)/ Npliss))
    est_lambda_PB_list.append(-np.log((Nplisssacc +1)/ Npliss))
    

    
    
    

In [ ]:
HMM_est_pliss_mu_b = []
HMM_est_pliss_sigma_b = []
HMM_est_sacc_mu_b = []
HMM_est_sacc_sigma_b = []
HMM_est_fix_mu_b = []
HMM_est_fix_sigma_b = []
HMM_est_D_list_b = []
HMM_est_lambda_SF_list_b = []
HMM_est_lambda_SP_list_b = []
HMM_est_lambda_FS_list_b = []
HMM_est_lambda_FP_list_b = []
HMM_est_lambda_PS_list_b = []
HMM_est_lambda_PF_list_b = []

HMM_logdS_fix_list_b = []
HMM_logdS_sacc_list_b = []
HMM_logdS_pliss_list_b = []

for i in range(len(lvS)):
    print(i)
    #############Pr Fixations ######################
    prdb = 0.2*np.exp(-est_lambda_DB_list[i])
    prdp = 0.2*np.exp(-est_lambda_DP_list[i])
    prdd = 1-prdb - prdp
    #############Pr Saccades########################
    prbd = 0.2*np.exp(-est_lambda_BD_list[i])
    prbp = 0.2*np.exp(-est_lambda_BP_list[i])
    prbb = 1-prbd - prbp
    #############Pr plissades ######################
    prpd = 0.2*np.exp(-est_lambda_PD_list[i])
    prpb = 0.2*np.exp(-est_lambda_PB_list[i])
    prpp = 1-prpb - prpd
    ################################################


    
    s1 = State(LogNormalDistribution(est_fix_mu[i], est_fix_sigma[i])) #fixations
    s2 = State(LogNormalDistribution(1.3*est_sacc_mu[i], est_sacc_sigma[i])) #saccades
    s3 = State(LogNormalDistribution(1.3*est_pliss_mu[i], est_pliss_sigma[i])) #plissades
    model = HiddenMarkovModel()
    model.add_states(s1, s2, s3)
    model.add_transition(model.start, s1, 0.4)
    model.add_transition(model.start, s2, 0.3)
    model.add_transition(model.start, s3, 0.3)
    model.add_transition(s1, s1, prdd)
    model.add_transition(s1, s2, prdb)
    model.add_transition(s1, s3, prdp)
    model.add_transition(s2, s1, prbd)
    model.add_transition(s2, s2, prbb)
    model.add_transition(s2, s3, prbp)
    model.add_transition(s3, s1, prpd)
    model.add_transition(s3, s2, prpb)
    model.add_transition(s3, s3, prpp)
    model.add_transition(s1, model.end, 0.4*len(lvS[i][lvS[i]>0]))
    model.add_transition(s2, model.end, 0.3*len(lvS[i][lvS[i]>0]))
    model.add_transition(s3, model.end, 0.3*len(lvS[i][lvS[i]>0]))
    model.bake()
    model.fit( [lvS[i][lvS[i]>0]] )
    temp_vel = lvS[i][lvS[i]>0]
    
    s1_mu = model.states[0].distribution.parameters[0]
    s2_mu = model.states[1].distribution.parameters[0]
    s3_mu = model.states[2].distribution.parameters[0]
    pos_array = np.argsort([s1_mu,s2_mu,s3_mu])
    pos_array2 = np.array([np.where(np.argsort([s1_mu,s2_mu,s3_mu]) ==0)[0][0], np.where(np.argsort([s1_mu,s2_mu,s3_mu]) ==1)[0][0],np.where(np.argsort([s1_mu,s2_mu,s3_mu]) ==2)[0][0]])
 
    HMM_est_fix_mu_b.append(model.states[pos_array[0]].distribution.parameters[0])
    HMM_est_fix_sigma_b.append(model.states[pos_array[0]].distribution.parameters[1])
    HMM_est_sacc_mu_b.append(model.states[pos_array[1]].distribution.parameters[0])
    HMM_est_sacc_sigma_b.append(model.states[pos_array[1]].distribution.parameters[1])
    HMM_est_pliss_mu_b.append(model.states[pos_array[2]].distribution.parameters[0])
    HMM_est_pliss_sigma_b.append(model.states[pos_array[2]].distribution.parameters[1])
    temp_HMM_bin_vec = np.swapaxes(model.viterbi(lvS[i][lvS[i]>0])[1],0,1)[0][1:-1]
    HMM_bin_vec = np.array([pos_array2[kk] for kk in temp_HMM_bin_vec])

        
        

    #Nfix = len(HMM_bin_vec) - np.sum(HMM_bin_vec)
    #Nsacc = np.sum(HMM_bin_vec)
    #Ntransi = int(np.sum(np.abs(HMM_bin_vec[1:] - HMM_bin_vec[:-1]))/2)
    #HMM_est_lambda_B_list.append(-np.log(1 - Ntransi / Nsacc))
    #HMM_est_lambda_D_list.append(-np.log(1 - Ntransi / Nfix))
    HMM_logdS_fix_list_b.append(temp_vel[HMM_bin_vec==0])
    HMM_logdS_sacc_list_b.append(temp_vel[HMM_bin_vec==1])
    HMM_logdS_pliss_list_b.append(temp_vel[HMM_bin_vec==2])


In [ ]:
    HMM_logdS_fix_list_b.append(temp_vel[HMM_bin_vec==0])
    HMM_logdS_sacc_list_b.append(temp_vel[HMM_bin_vec==1])
    HMM_logdS_pliss_list_b.append(temp_vel[HMM_bin_vec==2])

In [ ]:
len(np.hstack(vS))

In [ ]:
    temp_HMM_bin_vec = np.swapaxes(model.viterbi(lvS[i][lvS[i]>0])[1],0,1)[0][1:-1]
    HMM_bin_vec = np.array([pos_array2[kk] for kk in temp_HMM_bin_vec])

        
        

    #Nfix = len(HMM_bin_vec) - np.sum(HMM_bin_vec)
    #Nsacc = np.sum(HMM_bin_vec)
    #Ntransi = int(np.sum(np.abs(HMM_bin_vec[1:] - HMM_bin_vec[:-1]))/2)
    #HMM_est_lambda_B_list.append(-np.log(1 - Ntransi / Nsacc))
    #HMM_est_lambda_D_list.append(-np.log(1 - Ntransi / Nfix))
    HMM_logdS_fix_list_b.append(temp_vel[HMM_bin_vec==0])
    HMM_logdS_sacc_list_b.append(temp_vel[HMM_bin_vec==1])
    HMM_logdS_pliss_list_b.append(temp_vel[HMM_bin_vec==2])


In [ ]:
i=0
plt.hist(np.log(HMM_logdS_fix_list_b[i]),np.arange(-2,4,0.2),density=False);
plt.hist(np.log(HMM_logdS_sacc_list_b[i]),np.arange(-2,4,0.2),density=False,alpha=0.6);
plt.hist(np.log(HMM_logdS_pliss_list_b[i]),np.arange(-2,4,0.2),density=False,alpha=0.4);

In [ ]:
import scipy.signal
N=5
for i in range(len(detection_list)):
    lc_smooth_det = np.convolve(detection_list[i], np.ones(N), 'valid') / N
    plt.plot(lc_smooth_det)
    #minima = scipy.signal.argrelextrema(lc_smooth_det, np.less)
    minima = scipy.signal.find_peaks(lc_smooth_det, distance=14)[0]
    plt.vlines(minima,0.1,0.3,color='k')
    print(scipy.signal.argrelextrema(lc_smooth_det, np.less))
    plt.show()
    
    

In [ ]:
temp_minima

In [ ]:
scipy.signal.argrelextrema(lc_smooth_det,np.greater)[0]

In [ ]:
lc_smooth_det[28:33]

In [ ]:
len(lc_smooth_det)-1

In [ ]:
temp_minima[0]

In [ ]:
temp_maxima

In [ ]:
temp_minima

In [ ]:
minima[0][0]

In [ ]:
tau_list = np.power(2,np.arange(9))
len_vector = len(dS2_lag[0])
adj_r_square_lev = []
adj_r_square_int = []
params_int = []
params_lev = []

for part in range(0,len(participants)):
    initial_conditions = []
    for k in range(len_vector-2):
        blabla = first_estimate_ple_averages_inc(dS2_lag[part][k:k+3],tau_list)
        if blabla[1]:
            initial_conditions.append(blabla[0])


    if len(initial_conditions)!=0:

        best_params = optimize(dS2_lag[part],tau_list,initial_conditions[0],1000,50)
        curr_min = np.mean( (np.log2(dS2_lag[part]) - log2_moment_scaling(tau_list,best_params[0],best_params[1] ,best_params[2]))**2)
        best_params_index = 0
        opt_meu = 1
        for k in range(len(initial_conditions)):
            popt3 = optimize(dS2_lag[part],tau_list,initial_conditions[k],1000,50)
            new_min = np.mean((np.log2(dS2_lag[part]) - log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2]))**2 )
            if new_min != new_min:
                new_min = curr_min + 1000
            if new_min < curr_min:
                best_params = popt3
                curr_min = new_min
                best_params_index = k
                opt_meu = 1

            try:
                popt3, pcov = scipy.optimize.curve_fit(log2_moment_scaling, tau_list, np.log2(dS2_lag[part]),p0=initial_conditions[k], maxfev=500000)
                new_min = np.mean((np.log2(dS2_lag[part]) - log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2]))**2 )
                if new_min != new_min:
                    new_min = curr_min + 1000
                if new_min < curr_min:
                    best_params = popt3
                    curr_min = new_min
                    best_params_index = k
                    opt_meu = 0
            except RuntimeError:
                'a'
        print(best_params_index,opt_meu)


    if len(initial_conditions)==0:
        best_params = optimize(dS2_lag[part],tau_list,[-1,1,0.01],1000,20)
        curr_min = np.mean( (np.log2(dS2_lag[part]) - log2_moment_scaling(tau_list,best_params[0],best_params[1] ,best_params[2]))**2)
        best_params_index = -1
        opt_meu = 1

        try:
            popt3, pcov = scipy.optimize.curve_fit(log2_moment_scaling, tau_list, np.log2(dS2_lag[part]),p0=[-1,1,0.01], maxfev=50000000)
            new_min = np.mean((np.log2(dS2_lag[part]) - log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2]))**2 )

            if new_min != new_min:
                new_min = curr_min + 1000
            if new_min < curr_min:
                best_params = popt3
                curr_min = new_min
                best_params_index = k
                opt_meu = 0

        except RuntimeError:
            'a'

    print(best_params_index,opt_meu)
    
    params_int.append(best_params)
    


    

    reg2 = LinearRegression().fit(np.log2(np.array(tau_list)).reshape(-1, 1), np.log2(dS2_lag[part]))
    coef = reg2.coef_[0]
    intercept = reg2.intercept_
    
    params_lev.append([intercept,coef])
    
    adj_r_square_lev.append(adjusted_r_square(np.log2(dS2_lag[part]),np.arange(7)*coef + intercept,2))
    adj_r_square_int.append(adjusted_r_square(np.log2(dS2_lag[part]),log2_moment_scaling(tau_list,best_params[0],best_params[1] ,best_params[2]),3))
#popt2, pcov2 = scipy.optimize.curve_fit(exponential_fit, np.array(tau_list), np.array(ds_square_lag), maxfev=5000000)
#print(popt2)
#print(C1_new2(tau,g_alpha,g_beta,g_D,g_v),C2_new2(tau,g_alpha,g_beta,g_D,g_v),lambda2)
#plt.plot(Sergyi_expr_simp(np.arange(tau_list),*popt))

In [ ]:
dS2_lag[part]

In [ ]:
np.array(adj_r_square_int) - np.array(  adj_r_square_lev)

In [ ]:
#plt.hist(np.log10(np.array(adj_r_square_lev)[np.array(adj_r_square_int)>0.7]  / np.array(adj_r_square_int)[np.array(adj_r_square_int)>0.7]), np.arange(-0.1,0.1,0.01),color='dimgrey')
plt.hist(np.array(adj_r_square_int) - np.array(adj_r_square_lev), np.arange(-0.1,0.1,0.01),color='dimgrey')
plt.vlines(0, 0, 30,colors='k')

In [ ]:
plt.hist(np.log10(np.array(adj_r_square_lev)  / np.array(adj_r_square_int) ), np.arange(-1,1.1,0.01))

In [ ]:
log2_moment_scaling(tau_list,initial_conditions[0],initial_conditions[1] ,initial_conditions[2])

In [ ]:
np.log2(dS2_lag[part])

In [ ]:
adj_r_square_int

In [ ]:
adj_r_square_lev

In [ ]:
np.log2(dS2_lag[part])

In [ ]:
log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2])

In [ ]:
adjusted_r_square(np.log2(dS2_lag[part]),log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2]),3)

In [ ]:
adj_r_square_lev = []
adj_r_square_int = []
for i in range(60):
    print(i)
    if i !=33:
        tau_list = np.power(2,np.arange(7))
        popt3, pcov = scipy.optimize.curve_fit(log2_moment_scaling, tau_list, np.log2(dS2_lag[i]), maxfev=5000000)
        print(popt3)

        reg2 = LinearRegression().fit(np.log2(np.array(tau_list)).reshape(-1, 1), np.log2(dS2_lag[i]))
        coef = reg2.coef_[0]
        intercept = reg2.intercept_

        adj_r_square_lev.append(adjusted_r_square(np.log2(dS2_lag[i]),np.arange(7)*coef + intercept,2))
        adj_r_square_int.append(adjusted_r_square(np.log2(dS2_lag[i]),log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2]),3))

In [ ]:
gamma = np.log10(np.array(adj_r_square_lev)/ np.array(adj_r_square_int))
plt.hist(gamma,40)

In [ ]:
adj_r_square_lev_test = []
adj_r_square_int_test = []
adj_r_square_lev_control = []
adj_r_square_int_control = []
for i in range(60):
    print(i)
    if i !=33:
        if i%2 == 1:
            tau_list = np.power(2,np.arange(7))
            popt3, pcov = scipy.optimize.curve_fit(log2_moment_scaling, tau_list, np.log2(dS2_lag[i]), maxfev=5000000)
            print(popt3)

            reg2 = LinearRegression().fit(np.log2(np.array(tau_list)).reshape(-1, 1), np.log2(dS2_lag[i]))
            coef = reg2.coef_[0]
            intercept = reg2.intercept_

            adj_r_square_lev_control.append(adjusted_r_square(np.log2(dS2_lag[i]),np.arange(7)*coef + intercept,2))
            adj_r_square_int_control.append(adjusted_r_square(np.log2(dS2_lag[i]),log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2]),3))
        
        
        if i%2==0:
            tau_list = np.power(2,np.arange(7))
            popt3, pcov = scipy.optimize.curve_fit(log2_moment_scaling, tau_list, np.log2(dS2_lag[i]), maxfev=5000000)
            print(popt3)

            reg2 = LinearRegression().fit(np.log2(np.array(tau_list)).reshape(-1, 1), np.log2(dS2_lag[i]))
            coef = reg2.coef_[0]
            intercept = reg2.intercept_

            adj_r_square_lev_test.append(adjusted_r_square(np.log2(dS2_lag[i]),np.arange(7)*coef + intercept,2))
            adj_r_square_int_test.append(adjusted_r_square(np.log2(dS2_lag[i]),log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2]),3))

gamma_test = np.log10(np.array(adj_r_square_lev_test)/ np.array(adj_r_square_int_test))
gamma_control = np.log10(np.array(adj_r_square_lev_control)/ np.array(adj_r_square_int_control))

In [ ]:
plt.hist(gamma_test,np.arange(-0.05,0.05,0.0025),alpha=0.5)
plt.hist(gamma_control,np.arange(-0.05,0.05,0.0025),alpha=0.5)

In [ ]:
print(np.sum(gamma_test>=0))
print(np.sum(gamma_control>=0))

In [ ]:
np.mean(gamma_test)

In [ ]:
adj_r_square_lev = []
adj_r_square_int = []
for i in range(60):
    print(i)
    if i !=33:
        tau_list = np.power(2,np.arange(7))
        popt3, pcov = scipy.optimize.curve_fit(log2_moment_scaling, tau_list, np.log2(dS2_lag[i]), maxfev=5000000)
        print(popt3)

        reg2 = LinearRegression().fit(np.log2(np.array(tau_list)).reshape(-1, 1), np.log2(dS2_lag[i]))
        coef = reg2.coef_[0]
        intercept = reg2.intercept_

        adj_r_square_lev.append(adjusted_r_square(np.log2(dS2_lag[i]),np.arange(7)*coef + intercept,2))
        adj_r_square_int.append(adjusted_r_square(np.log2(dS2_lag[i]),log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2]),3))

        plt.plot(np.log2(dS2_lag[i]),'ks',label='eye gaze trajectory',alpha=0.5)
        plt.plot(log2_moment_scaling(tau_list,popt3[0],popt3[1] ,popt3[2])  ,label='intermittent fit',c='tab:blue')
        plt.plot(np.arange(7)*coef + intercept  ,label='Lévy walk fit',c='tab:red')
        plt.xlabel(r'$log_2(\tau)$')
        plt.ylabel(r'$log \|\|u^2\|\|$')
        plt.legend()
        plt.savefig('Fig4-test_old_data_P' + str(i) + '.png',dpi=300)
        plt.close()
        
#plt.plot(np.log2(moment_scaling(np.array(tau_list),popt[0],popt[1],popt[2])))

In [ ]:
#SAVE DATA IN INDEPENDENT SERIES. 
#THE ORIGINAL DATA DOES NOT SEPARATE BETWEEN TRIALS OF EACH PARTICIPANT AND INCLUES DE DATA-POINTS OF THE CALLIBRATION.


In [ ]:
selected_rows = rmissingvaluecol(data[0],100)
NaN_percentage = []
all_series = []
for i in range(len(data)):
    if i % 2 == 0:
        group = 'dataset_III'
    if i % 2 == 1:
        group = 'dataset_II'
    cal_end = data[i].loc[data[i]['Event'] == 'Eye tracker Calibration end'].index #END OF CALLIBRATION
    rec_end = data[i].loc[data[i]['Event'] == 'RecordingEnd'].index #END OF TRIAL
    count = 0
    for j in range(len(cal_end)):
        lnew_series = data[i].iloc[cal_end[j]: rec_end[j+count]]
        new_series = lnew_series[selected_rows].copy()  #TIME-SERIES OF ONE INDIVIDUAL TRIAL
        length = rec_end[j+count] - cal_end[j]
        while length<0:
            count +=1
            new_series = data[i].iloc[cal_end[j]: rec_end[j+count]]
            length = rec_end[j+count] - cal_end[j]
            
##################################################################
        if length > 600: #WE WILL NOT CONSIDER TRIALS WITH LESS THAN 600 POINTS
            im_name =  which_image(data[i]['Presented Stimulus name'].iloc[cal_end[j]: rec_end[j+count]])
            nan_rat = 1- new_series['Gaze point X'].dropna().shape[0] / new_series['Gaze point X'].shape[0]
            if nan_rat < 0.6:
                #new_series.dropna(how='all', axis=1).to_csv('EyeT_group_'+group+ '_image_name_' + im_name +'_'+'participant_'+str(i)+'_'+'trial'+str(j)+'.csv')

                all_series.append(new_series) #LIST OF TIME-SERIES OF TRIALS
                NaN_percentage.append(new_series['Gaze point X'].dropna().shape[0] / new_series['Gaze point X'].shape[0])

In [ ]:
#SERIES LENGTH

In [ ]:
lengths = []
for j in range(len(all_series)):
    lengths.append(len(all_series[j]['Gaze point X'].dropna()))
#plt.hist(lengths,np.arange(4000,12000,500))
a = plt.hist(lengths,np.arange(0,30000,1200))
np.savetxt("hist_length_time_series.txt",np.transpose([np.arange(600,30000-600,1200),np.array(a[0])]))
plt.xlabel('data-points in each time-series')
plt.ylabel('frequency')
plt.title('histogram: length of each time-series')
plt.savefig('data_points_frequency.png',dpi = 500)

In [ ]:
## PERCENTAGE OF MISSING VALUES

In [ ]:
a = plt.hist(np.ones(len(NaN_percentage))- np.array(NaN_percentage),np.arange(0,0.6,0.025))
np.savetxt("NaN_ratio_frequencies_x_axis.txt",a[1])
np.savetxt("NaN_ratio_frequencies.txt",np.transpose([np.arange(0.0125,0.6-0.0125,0.025),np.array(a[0])/np.sum(np.array(a[0]))]))
plt.title('Histogram of NaN ratios')
plt.xlabel('NaN ratio')
plt.ylabel('Frequency in our database')
plt.savefig('hist_NaN',dpi=500)

In [ ]:
durations_all_series= []
for j in range(len(all_series)):
    new_array = []
    current_array = np.array(all_series[j].dropna(subset=['Gaze point X'])['Eye movement type'])
    for i in range(len(current_array)):
        if current_array[i] == 'Saccade':
            new_array.append(1)
        elif current_array[i] == 'Fixation':
            new_array.append(0)
        elif current_array[i]  == 'Unclassified':
            new_array.append(-1)
        elif current_array[i] == 'EyesNotFound':
            new_array.append(-2)
    durations_all_series.append(new_array)
    

In [ ]:
#CHECKING THE LABELS OF SACCADE AND FIXATION

In [ ]:
total_fix = 0
total_sac = 0
total_unclassified = 0
for j in range(len(all_series)):
    current_array = np.array(all_series[j].dropna(subset=['Gaze point X'])['Eye movement type'])
    for i in range(len(all_series[j].dropna(subset=['Gaze point X'])['Eye movement type'])):
        if current_array[i] == 'Saccade':
            total_sac +=1
        elif current_array[i] == 'Fixation':
            total_fix +=1
        elif current_array[i] == 'Unclassified':
            total_unclassified +=1    

In [ ]:
print(total_fix)
print(total_sac)
print(total_unclassified)
print(total_fix+total_sac+total_unclassified)

In [ ]:
#CONSECUTIVE FIXATIONS

In [ ]:
total_cons_fix = []
total_cons_sacc = []
for j in range(len(all_series)-1):
    consecutive_fix = consecutive_sac(durations_all_series[j] , 0)  
    total_cons_fix = total_cons_fix + list(consecutive_fix[0])
    consecutive_sacc = consecutive_sac(durations_all_series[j],1)  
    total_cons_sac = total_cons_sac + list(consecutive_sacc)


In [ ]:
means = []
total_cons_fix = []
for j in range(len(all_series)-1):
    consecutive_fix = consecutive_sac(durations_all_series[j],0)  
    total_cons_fix = total_cons_fix + list(consecutive_fix[0])
    band = 1.06*np.std(consecutive_fix[0])*(len(consecutive_fix[0]))**(-0.2)
    kde = KernelDensity(kernel='gaussian', bandwidth=band).fit(np.array(consecutive_fix[0]).reshape(-1, 1))
    log_dens = np.exp(kde.score_samples(np.arange(0,200).reshape(-1, 1)))
    plt.plot(log_dens)
    plt.xlabel('consecutive fixations')
    plt.ylabel('probability density')
    means.append(np.mean(consecutive_fix[0]))
plt.savefig('fix_duration.png',dpi = 500)
plt.show()


In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth=band).fit(np.array(total_cons_fix).reshape(-1, 1))
log_dens_fix = np.exp(kde.score_samples(np.arange(0,200).reshape(-1, 1)))
a = np.transpose([np.arange(0,200),log_dens_fix])
#np.savetxt("total_fixations_length_pdf_x_axis"+".txt",np.arange(200))
plt.plot(log_dens)
plt.xlabel('consecutive fixations')
plt.ylabel('probability density')
plt.title('Total Fixation Length pdf')

In [ ]:
#CONSECUTIVE SACCADES

In [ ]:
total_cons_sac = []
for j in range(len(all_series)):
    consecutive_fix = consecutive_sac(durations_all_series[j],1)   
    total_cons_sac = total_cons_sac + list(consecutive_fix[0])
    band = 2*1.06*np.std(consecutive_fix[0])*(len(consecutive_fix[0]))**(-0.2)
    kde = KernelDensity(kernel='gaussian', bandwidth=band).fit(np.array(consecutive_fix[0]).reshape(-1, 1))
    log_dens = np.exp(kde.score_samples(np.arange(0,20).reshape(-1, 1)))
    plt.plot(np.arange(0,20),log_dens)
    plt.xlabel('consecutive saccades')
    plt.ylabel('probability density')
    np.savetxt("saccades_length_pdf"+str(j) + ".txt",log_dens)
    np.savetxt("saccades_length_x_axis"+str(j) + ".txt",np.arange(0,20))
    means.append(np.mean(consecutive_fix[0]))
plt.savefig('sac_duration.png',dpi = 500)
plt.show()   

In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth=band).fit(np.array(total_cons_sac).reshape(-1, 1))
log_dens_sac = np.exp(kde.score_samples(np.arange(0,20).reshape(-1, 1)))
bb = np.transpose([np.arange(1,21),log_dens_sac])
np.savetxt("total_sac_length_pdf"+ ".txt",bb)
plt.plot(log_dens)
plt.xlabel('consecutive saccades')
plt.ylabel('probability density')
plt.title('Total Saccade Length pdf')

In [ ]:
#Ratio of Saccades and Fixations

In [ ]:
consecutive_sacc_array= []
percentage_sacc = []

for j in range(len(all_series)):
    a = consecutive_sac(durations_all_series[j],1)
    consecutive_sacc_array.append(a[0]) 
    percentage_sacc.append(np.array(a[0]).sum()/ (all_series[j]['Gaze point X'].dropna().shape[0]))
    
a = plt.hist(percentage_sacc,np.arange(0,1.02,0.02))
plt.xlabel('saccade to series length ratio')
plt.ylabel('frequency')
plt.title('')
plt.savefig('./histogram saccade-ratio.png',dpi = 500)

In [ ]:
consecutive_fix_array= []
percentage_fix = []

for j in range(len(all_series)):
    a = consecutive_sac(durations_all_series[j],0)
    consecutive_fix_array.append(a[0]) 
    percentage_fix.append(np.array(a[0]).sum()/ (all_series[j]['Gaze point X'].dropna().shape[0]))
    
a = plt.hist(percentage_fix,np.arange(0,1,0.04))
np.savetxt("percentage_fix.txt",np.transpose([np.arange(0.02,0.98,0.04),np.array(a[0])/np.sum(np.array(a[0]))]))
plt.xlabel('fixation to series length ratio')
plt.ylabel('frequency')
#plt.title('histogram: saccade-ratio')
plt.savefig('./histogram fixation-ratio.png',dpi = 500)

In [ ]:
consecutive_unclassified_array= []
percentage_unclassified = []

for j in range(len(all_series)):
    a = consecutive_sac(durations_all_series[j],-1)
    consecutive_unclassified_array.append(a[0]) 
    percentage_unclassified.append(np.array(a[0]).sum()/ (all_series[j]['Gaze point X'].dropna().shape[0]))
    
a = plt.hist(percentage_unclassified,np.arange(0,1,0.04))
np.savetxt("percentage_unclassified.txt",np.transpose([np.arange(0.02,0.98,0.04),np.array(a[0])/np.sum(np.array(a[0]))]))
plt.xlabel(" 'Unclassified'/'Eyes not found' to series length ratio")
plt.ylabel('frequency')
plt.savefig('./histogram unclassified-ratio.png',dpi = 500)